# 🐍🗄️ MÓDULO 3.2: PYTHON + SQLITE
## 📊 Maestría en Python - Fase 3: Integración de Datos

---

### 🎯 OBJETIVOS DE APRENDIZAJE

Al completar este módulo serás capaz de:
- ✅ Integrar Python con SQLite de forma profesional
- ✅ Implementar patrones DAO y Repository
- ✅ Crear sistemas de monitoreo industrial en tiempo real
- ✅ Automatizar reportes y análisis de datos
- ✅ Usar Pandas para análisis híbrido SQL-Python
- ✅ Implementar seguridad y mejores prácticas

### 📋 TEMARIO DETALLADO

1. **Conexión Python-SQLite** (sqlite3, context managers)
2. **Operaciones CRUD** (Create, Read, Update, Delete)
3. **Patrones de Diseño** (DAO, Repository, Factory)
4. **Pandas + SQL** (Análisis híbrido de datos)
5. **Sistema de Monitoreo Industrial** (Tiempo real, alertas)
6. **Automatización de Reportes** (Scheduling, exports)
7. **Optimización y Seguridad** (Pool conexiones, validación)
8. **Proyecto Integrador** (Dashboard industrial completo)

---

## 🔌 1. CONEXIÓN PYTHON-SQLITE

### 🤝 Integrando los Mundos SQL y Python

Python incluye **sqlite3** de forma nativa, permitiendo una integración perfecta con bases de datos. Aprenderemos desde conexiones básicas hasta patrones profesionales.

### 🏗️ Fundamentos de Conexión

In [1]:
# 🚀 CONFIGURACIÓN INICIAL - Conexión Básica Python-SQLite
import sqlite3
import pandas as pd
from datetime import datetime, date
from contextlib import contextmanager
from typing import List, Dict, Any, Optional
import json
import random
from pathlib import Path

print("✅ Librerías importadas exitosamente")
print(f"📅 Sesión iniciada: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Versión Python con sqlite3 integrado")

✅ Librerías importadas exitosamente
📅 Sesión iniciada: 2025-07-05 08:59:04
🐍 Versión Python con sqlite3 integrado


In [2]:
# 🔗 CONEXIÓN BÁSICA A SQLITE
def conexion_basica_demo():
    """Demostración de conexión básica"""
    
    # Crear/conectar a base de datos
    conn = sqlite3.connect('empresa_industrial.db')
    print("✅ Conexión establecida a 'empresa_industrial.db'")
    
    # Configurar para resultados como diccionarios
    conn.row_factory = sqlite3.Row
    print("🔧 Configurado row_factory para resultados tipo diccionario")
    
    # Crear cursor para ejecutar comandos
    cursor = conn.cursor()
    
    # Verificar versión de SQLite
    cursor.execute("SELECT sqlite_version()")
    version = cursor.fetchone()[0]
    print(f"📊 Versión SQLite: {version}")
    
    # Listar tablas existentes
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tablas = cursor.fetchall()
    print(f"📋 Tablas existentes: {[tabla[0] for tabla in tablas]}")
    
    # ¡IMPORTANTE! Cerrar conexión
    conn.close()
    print("🔒 Conexión cerrada correctamente")
    
    return version

# Ejecutar demostración
version_sqlite = conexion_basica_demo()

print("\n" + "="*50)
print("🧠 CONCEPTOS CLAVE:")
print("✅ sqlite3.connect() - Establece conexión")
print("✅ row_factory - Configura formato de resultados")
print("✅ cursor() - Ejecuta comandos SQL")
print("✅ conn.close() - Libera recursos")
print("="*50)

✅ Conexión establecida a 'empresa_industrial.db'
🔧 Configurado row_factory para resultados tipo diccionario
📊 Versión SQLite: 3.49.1
📋 Tablas existentes: []
🔒 Conexión cerrada correctamente

🧠 CONCEPTOS CLAVE:
✅ sqlite3.connect() - Establece conexión
✅ row_factory - Configura formato de resultados
✅ cursor() - Ejecuta comandos SQL
✅ conn.close() - Libera recursos


### 🛡️ Context Managers - Gestión Segura de Conexiones

Los **context managers** garantizan que las conexiones se cierren automáticamente, incluso si ocurre un error. Esto es **crítico** en aplicaciones industriales.

In [7]:
# 🛡️ CONTEXT MANAGER PROFESIONAL
@contextmanager
def obtener_conexion_segura(db_path: str):
    """Context manager para manejo seguro de conexiones"""
    conn = None
    try:
        print(f"🔗 Estableciendo conexión a {db_path}")
        conn = sqlite3.connect(db_path)
        conn.row_factory = sqlite3.Row  # Resultados como diccionarios
        print("✅ Conexión establecida y configurada")
        yield conn
    except sqlite3.Error as e:
        print(f"❌ Error en base de datos: {e}")
        if conn:
            conn.rollback()
            print("🔄 Rollback ejecutado")
        raise
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        if conn:
            conn.rollback()
        raise
    finally:
        if conn:
            conn.close()
            print("🔒 Conexión cerrada automáticamente")

# Demostración del context manager
def demo_context_manager():
    """Demuestra el uso del context manager"""
    
    print("🚀 DEMOSTRACIÓN CONTEXT MANAGER")
    print("-" * 40)
    
    # Uso normal (exitoso)
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT 'Operación exitosa' as mensaje")
            resultado = cursor.fetchone()
            print(f"📋 Resultado: {resultado['mensaje']}")
            # La conexión se cierra automáticamente aquí
    except Exception as e:
        print(f"Error capturado: {e}")
    
    print("\n🧪 Simulando error para mostrar rollback automático:")
    
    # Simulación de error
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            # SQL inválido para forzar error
            cursor.execute("SELECT FROM tabla_inexistente")
    except Exception as e:
        print(f"🎯 Error capturado y manejado: {type(e).__name__}")
    
    print("\n✅ Demostración completada - Conexiones manejadas seguramente")

# Ejecutar demostración
demo_context_manager()

print("\n" + "="*50)
print("🧠 VENTAJAS DEL CONTEXT MANAGER:")
print("✅ Cierre automático de conexiones")
print("✅ Rollback automático en errores")
print("✅ Manejo consistente de excepciones")
print("✅ Código más limpio y profesional")
print("="*50)

🚀 DEMOSTRACIÓN CONTEXT MANAGER
----------------------------------------
🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada
📋 Resultado: Operación exitosa
🔒 Conexión cerrada automáticamente

🧪 Simulando error para mostrar rollback automático:
🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada
❌ Error en base de datos: near "FROM": syntax error
🔄 Rollback ejecutado
🔒 Conexión cerrada automáticamente
🎯 Error capturado y manejado: OperationalError

✅ Demostración completada - Conexiones manejadas seguramente

🧠 VENTAJAS DEL CONTEXT MANAGER:
✅ Cierre automático de conexiones
✅ Rollback automático en errores
✅ Manejo consistente de excepciones
✅ Código más limpio y profesional


## 📊 2. OPERACIONES CRUD CON PYTHON

### 🔧 Create, Read, Update, Delete

Implementaremos las operaciones fundamentales CRUD usando Python, integrando los conocimientos SQL del módulo anterior con patrones de programación profesionales.

In [4]:
# 🏗️ CONFIGURACIÓN DE TABLAS PARA CRUD
def inicializar_sistema_empleados():
    """Inicializa el sistema de gestión de empleados"""
    
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        
        # Crear tabla empleados con todas las validaciones
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS empleados (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nombre TEXT NOT NULL CHECK(length(nombre) > 0),
                apellido TEXT NOT NULL CHECK(length(apellido) > 0),
                email TEXT UNIQUE NOT NULL CHECK(email LIKE '%@%'),
                telefono TEXT CHECK(length(telefono) >= 10),
                salario REAL CHECK(salario > 0),
                departamento TEXT NOT NULL,
                fecha_ingreso DATE NOT NULL,
                años_experiencia INTEGER CHECK(años_experiencia >= 0),
                activo BOOLEAN DEFAULT 1,
                fecha_creacion DATETIME DEFAULT CURRENT_TIMESTAMP,
                fecha_actualizacion DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Crear tabla de auditoría
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS auditoria_empleados (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                empleado_id INTEGER,
                operacion TEXT NOT NULL,
                datos_anteriores TEXT,
                datos_nuevos TEXT,
                usuario TEXT DEFAULT 'sistema',
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Crear índices para optimización
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_empleados_email ON empleados(email)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_empleados_departamento ON empleados(departamento)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_empleados_activo ON empleados(activo)')
        
        conn.commit()
        print("✅ Sistema de empleados inicializado")
        print("📋 Tablas creadas: empleados, auditoria_empleados")
        print("🚀 Índices optimizados creados")

# Inicializar sistema
inicializar_sistema_empleados()

# Verificar estructura de la tabla
with obtener_conexion_segura('empresa_industrial.db') as conn:
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(empleados)")
    columnas = cursor.fetchall()
    
    print("\n📊 ESTRUCTURA DE LA TABLA EMPLEADOS:")
    print("-" * 60)
    for col in columnas:
        print(f"📄 {col['name']:<20} | {col['type']:<10} | {col['notnull']} | {col['dflt_value']}")
    
    print(f"\n🎯 Total de columnas: {len(columnas)}")

🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada
✅ Sistema de empleados inicializado
📋 Tablas creadas: empleados, auditoria_empleados
🚀 Índices optimizados creados
🔒 Conexión cerrada automáticamente
🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada

📊 ESTRUCTURA DE LA TABLA EMPLEADOS:
------------------------------------------------------------
📄 id                   | INTEGER    | 0 | None
📄 nombre               | TEXT       | 1 | None
📄 apellido             | TEXT       | 1 | None
📄 email                | TEXT       | 1 | None
📄 telefono             | TEXT       | 0 | None
📄 salario              | REAL       | 0 | None
📄 departamento         | TEXT       | 1 | None
📄 fecha_ingreso        | DATE       | 1 | None
📄 años_experiencia     | INTEGER    | 0 | None
📄 activo               | BOOLEAN    | 0 | 1
📄 fecha_creacion       | DATETIME   | 0 | CURRENT_TIMESTAMP
📄 fecha_actualizacion  | DATETIME   | 0 | CURRENT_TIM

### ➕ CREATE - Inserción de Datos

Implementaremos inserción de datos con validación, manejo de errores y auditoría.

### ✏️ UPDATE - Actualización de Datos

Implementaremos actualizaciones con auditoría completa y validación de cambios.

In [11]:
# ➕ OPERACIONES CREATE (INSERCIÓN)
def crear_empleado(datos_empleado: Dict[str, Any]) -> Optional[int]:
    """Crea un nuevo empleado con validación completa"""
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Validar email único
            cursor.execute("SELECT COUNT(*) FROM empleados WHERE email = ?", (datos_empleado['email'],))
            if cursor.fetchone()[0] > 0:
                print(f"❌ Error: Email {datos_empleado['email']} ya existe")
                return None
            
            # Insertar empleado
            cursor.execute('''
                INSERT INTO empleados (nombre, apellido, email, telefono, salario, 
                                     departamento, fecha_ingreso, años_experiencia)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                datos_empleado['nombre'],
                datos_empleado['apellido'],
                datos_empleado['email'],
                datos_empleado.get('telefono', ''),
                datos_empleado['salario'],
                datos_empleado['departamento'],
                datos_empleado['fecha_ingreso'],
                datos_empleado.get('años_experiencia', 0)
            ))
            
            empleado_id = cursor.lastrowid
            
            # Registrar auditoría
            cursor.execute('''
                INSERT INTO auditoria_empleados (empleado_id, operacion, datos_nuevos)
                VALUES (?, 'CREATE', ?)
            ''', (empleado_id, json.dumps(datos_empleado)))
            
            conn.commit()
            print(f"✅ Empleado creado exitosamente con ID: {empleado_id}")
            return empleado_id
            
    except sqlite3.IntegrityError as e:
        print(f"❌ Error de integridad: {e}")
        return None
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        return None

def crear_empleados_masivo(lista_empleados: List[Dict[str, Any]]) -> List[int]:
    """Crea múltiples empleados en una transacción"""
    
    empleados_creados = []
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            for empleado in lista_empleados:
                try:
                    cursor.execute('''
                        INSERT INTO empleados (nombre, apellido, email, telefono, salario, 
                                             departamento, fecha_ingreso, años_experiencia)
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        empleado['nombre'], empleado['apellido'], empleado['email'],
                        empleado.get('telefono', ''), empleado['salario'],
                        empleado['departamento'], empleado['fecha_ingreso'],
                        empleado.get('años_experiencia', 0)
                    ))
                    
                    empleado_id = cursor.lastrowid
                    empleados_creados.append(empleado_id)
                    
                    # Auditoría
                    cursor.execute('''
                        INSERT INTO auditoria_empleados (empleado_id, operacion, datos_nuevos)
                        VALUES (?, 'CREATE_BULK', ?)
                    ''', (empleado_id, json.dumps(empleado)))
                    
                except sqlite3.IntegrityError as e:
                    print(f"⚠️ Error al crear {empleado['email']}: {e}")
                    continue
            
            conn.commit()
            print(f"✅ Creación masiva completada: {len(empleados_creados)} empleados")
            return empleados_creados
            
    except Exception as e:
        print(f"❌ Error en creación masiva: {e}")
        return []

# 🧪 PRUEBAS DE INSERCIÓN
print("🧪 PROBANDO OPERACIONES CREATE")
print("=" * 50)

# Empleado individual
empleado_prueba = {
    'nombre': 'Carlos',
    'apellido': 'Rodríguez',
    'email': 'carlos.rodriguez@industrial.com',
    'telefono': '555-0001',
    'salario': 75000.00,
    'departamento': 'Automatización',
    'fecha_ingreso': '2024-01-15',
    'años_experiencia': 8
}

id_carlos = crear_empleado(empleado_prueba)

# Empleados masivos
empleados_masivos = [
    {
        'nombre': 'María', 'apellido': 'González', 
        'email': 'maria.gonzalez@industrial.com',
        'salario': 68000.00, 'departamento': 'Ingeniería',
        'fecha_ingreso': '2023-03-20', 'años_experiencia': 5
    },
    {
        'nombre': 'Juan', 'apellido': 'Pérez',
        'email': 'juan.perez@industrial.com', 
        'salario': 82000.00, 'departamento': 'Automatización',
        'fecha_ingreso': '2022-07-10', 'años_experiencia': 12
    },
    {
        'nombre': 'Ana', 'apellido': 'López',
        'email': 'ana.lopez@industrial.com',
        'salario': 59000.00, 'departamento': 'Calidad',
        'fecha_ingreso': '2024-02-01', 'años_experiencia': 3
    }
]

ids_creados = crear_empleados_masivo(empleados_masivos)

print(f"\n📊 RESUMEN DE CREACIÓN:")
print(f"👤 Empleado individual: ID {id_carlos}")
print(f"👥 Empleados masivos: {len(ids_creados)} creados")
print(f"🎯 IDs generados: {ids_creados}")

# Verificar total de empleados
with obtener_conexion_segura('empresa_industrial.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM empleados WHERE activo = 1")
    total = cursor.fetchone()[0]
    print(f"\n📈 Total empleados activos en sistema: {total}")

🧪 PROBANDO OPERACIONES CREATE
🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada
❌ Error en base de datos: CHECK constraint failed: length(telefono) >= 10
🔄 Rollback ejecutado
🔒 Conexión cerrada automáticamente
❌ Error de integridad: CHECK constraint failed: length(telefono) >= 10
🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada
⚠️ Error al crear maria.gonzalez@industrial.com: CHECK constraint failed: length(telefono) >= 10
⚠️ Error al crear juan.perez@industrial.com: CHECK constraint failed: length(telefono) >= 10
⚠️ Error al crear ana.lopez@industrial.com: CHECK constraint failed: length(telefono) >= 10
✅ Creación masiva completada: 0 empleados
🔒 Conexión cerrada automáticamente

📊 RESUMEN DE CREACIÓN:
👤 Empleado individual: ID None
👥 Empleados masivos: 0 creados
🎯 IDs generados: []
🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada

📈 Total empleados activos en sistema: 0
🔒 Con

### 🔍 READ - Consulta de Datos

Implementaremos diferentes patrones de consulta, desde búsquedas simples hasta análisis complejos.

In [10]:
# 🔍 OPERACIONES READ (CONSULTA)
def obtener_empleado_por_id(empleado_id: int) -> Optional[Dict[str, Any]]:
    """Obtiene un empleado específico por ID"""
    
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT * FROM empleados 
            WHERE id = ? AND activo = 1
        """, (empleado_id,))
        
        row = cursor.fetchone()
        return dict(row) if row else None

def buscar_empleados(filtros: Dict[str, Any] = None, orden: str = 'apellido') -> List[Dict[str, Any]]:
    """Búsqueda avanzada de empleados con filtros dinámicos"""
    
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        
        # Construir query dinámico
        where_clauses = ["activo = 1"]
        valores = []
        
        if filtros:
            for campo, valor in filtros.items():
                if isinstance(valor, str) and '%' in valor:
                    where_clauses.append(f"{campo} LIKE ?")
                elif isinstance(valor, dict) and 'min' in valor and 'max' in valor:
                    where_clauses.append(f"{campo} BETWEEN ? AND ?")
                    valores.extend([valor['min'], valor['max']])
                    continue
                elif isinstance(valor, list):
                    where_clauses.append(f"{campo} IN ({','.join(['?'] * len(valor))})")
                    valores.extend(valor)
                    continue
                else:
                    where_clauses.append(f"{campo} = ?")
                valores.append(valor)
        
        where_sql = " AND ".join(where_clauses)
        query = f"""
            SELECT * FROM empleados 
            WHERE {where_sql}
            ORDER BY {orden}
        """
        
        cursor.execute(query, valores)
        return [dict(row) for row in cursor.fetchall()]

def obtener_estadisticas_empleados() -> Dict[str, Any]:
    """Obtiene estadísticas completas del sistema"""
    
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        
        # Estadísticas generales
        cursor.execute("""
            SELECT 
                COUNT(*) as total_empleados,
                ROUND(AVG(salario), 2) as salario_promedio,
                MIN(salario) as salario_minimo,
                MAX(salario) as salario_maximo,
                SUM(salario) as nomina_total,
                ROUND(AVG(años_experiencia), 1) as experiencia_promedio
            FROM empleados WHERE activo = 1
        """)
        
        stats_generales = dict(cursor.fetchone())
        
        # Estadísticas por departamento
        cursor.execute("""
            SELECT 
                departamento,
                COUNT(*) as empleados,
                ROUND(AVG(salario), 2) as salario_promedio,
                SUM(salario) as costo_departamento
            FROM empleados 
            WHERE activo = 1
            GROUP BY departamento
            ORDER BY empleados DESC
        """)
        
        stats_departamento = [dict(row) for row in cursor.fetchall()]
        
        # Distribución por experiencia
        cursor.execute("""
            SELECT 
                CASE 
                    WHEN años_experiencia <= 2 THEN 'Junior (0-2 años)'
                    WHEN años_experiencia <= 5 THEN 'Semi-Senior (3-5 años)'
                    WHEN años_experiencia <= 10 THEN 'Senior (6-10 años)'
                    ELSE 'Experto (11+ años)'
                END as nivel,
                COUNT(*) as cantidad,
                ROUND(AVG(salario), 2) as salario_promedio
            FROM empleados 
            WHERE activo = 1
            GROUP BY nivel
            ORDER BY cantidad DESC
        """)
        
        distribucion_experiencia = [dict(row) for row in cursor.fetchall()]
        
        return {
            'generales': stats_generales,
            'por_departamento': stats_departamento,
            'por_experiencia': distribucion_experiencia,
            'timestamp': datetime.now().isoformat()
        }

# 🧪 PRUEBAS DE CONSULTA
print("🧪 PROBANDO OPERACIONES READ")
print("=" * 50)

# Consulta por ID
if id_carlos:
    empleado = obtener_empleado_por_id(id_carlos)
    if empleado:
        print(f"👤 EMPLEADO ID {id_carlos}:")
        print(f"   Nombre: {empleado['nombre']} {empleado['apellido']}")
        print(f"   Email: {empleado['email']}")
        print(f"   Departamento: {empleado['departamento']}")
        print(f"   Salario: ${empleado['salario']:,.2f}")

# Búsqueda por departamento
print("\n🔍 EMPLEADOS DE AUTOMATIZACIÓN:")
empleados_auto = buscar_empleados({'departamento': 'Automatización'})
for emp in empleados_auto:
    print(f"   • {emp['nombre']} {emp['apellido']} - ${emp['salario']:,.2f}")

# Búsqueda por rango salarial
print("\n💰 EMPLEADOS CON SALARIO $60K - $80K:")
empleados_rango = buscar_empleados({'salario': {'min': 60000, 'max': 80000}})
for emp in empleados_rango:
    print(f"   • {emp['nombre']} {emp['apellido']} ({emp['departamento']}) - ${emp['salario']:,.2f}")

# Búsqueda con LIKE
print("\n🔎 EMPLEADOS CON NOMBRES QUE CONTIENEN 'ar':")
empleados_like = buscar_empleados({'nombre': '%ar%'})
for emp in empleados_like:
    print(f"   • {emp['nombre']} {emp['apellido']}")

# Estadísticas completas
print("\n📊 ESTADÍSTICAS DEL SISTEMA:")
stats = obtener_estadisticas_empleados()

print("\n📈 GENERALES:")
gen = stats['generales']
print(f"   👥 Total empleados: {gen['total_empleados']}")
print(f"   💰 Salario promedio: ${gen['salario_promedio']:,.2f}")
print(f"   📊 Rango salarial: ${gen['salario_minimo']:,.2f} - ${gen['salario_maximo']:,.2f}")
print(f"   💸 Nómina total: ${gen['nomina_total']:,.2f}")
print(f"   ⏱️ Experiencia promedio: {gen['experiencia_promedio']} años")

print("\n🏢 POR DEPARTAMENTO:")
for dept in stats['por_departamento']:
    print(f"   • {dept['departamento']}: {dept['empleados']} empleados, promedio ${dept['salario_promedio']:,.2f}")

print("\n🎯 POR NIVEL DE EXPERIENCIA:")
for nivel in stats['por_experiencia']:
    print(f"   • {nivel['nivel']}: {nivel['cantidad']} empleados, promedio ${nivel['salario_promedio']:,.2f}")

🧪 PROBANDO OPERACIONES READ

🔍 EMPLEADOS DE AUTOMATIZACIÓN:
🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada
🔒 Conexión cerrada automáticamente

💰 EMPLEADOS CON SALARIO $60K - $80K:
🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada
🔒 Conexión cerrada automáticamente

🔎 EMPLEADOS CON NOMBRES QUE CONTIENEN 'ar':
🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada
🔒 Conexión cerrada automáticamente

📊 ESTADÍSTICAS DEL SISTEMA:
🔗 Estableciendo conexión a empresa_industrial.db
✅ Conexión establecida y configurada
🔒 Conexión cerrada automáticamente

📈 GENERALES:
   👥 Total empleados: 0


TypeError: unsupported format string passed to NoneType.__format__

In [ ]:
# ✏️ OPERACIONES UPDATE (ACTUALIZACIÓN)
def actualizar_empleado(empleado_id: int, datos_nuevos: Dict[str, Any]) -> bool:
    """Actualiza un empleado con auditoría completa"""
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Obtener datos actuales para auditoría
            cursor.execute("SELECT * FROM empleados WHERE id = ? AND activo = 1", (empleado_id,))
            empleado_actual = cursor.fetchone()
            
            if not empleado_actual:
                print(f"❌ Empleado ID {empleado_id} no encontrado")
                return False
            
            datos_anteriores = dict(empleado_actual)
            
            # Construir query dinámico para actualización
            campos_actualizacion = []
            valores = []
            
            for campo, valor in datos_nuevos.items():
                if campo in ['id', 'fecha_creacion']:  # Campos protegidos
                    continue
                campos_actualizacion.append(f"{campo} = ?")
                valores.append(valor)
            
            if not campos_actualizacion:
                print("⚠️ No hay campos válidos para actualizar")
                return False
            
            # Agregar fecha de actualización
            campos_actualizacion.append("fecha_actualizacion = CURRENT_TIMESTAMP")
            valores.append(empleado_id)
            
            query = f"""
                UPDATE empleados 
                SET {', '.join(campos_actualizacion)}
                WHERE id = ? AND activo = 1
            """
            
            cursor.execute(query, valores)
            
            if cursor.rowcount == 0:
                print(f"❌ No se pudo actualizar empleado ID {empleado_id}")
                return False
            
            # Registrar auditoría
            cursor.execute("""
                INSERT INTO auditoria_empleados (empleado_id, operacion, datos_anteriores, datos_nuevos)
                VALUES (?, 'UPDATE', ?, ?)
            """, (empleado_id, json.dumps(datos_anteriores), json.dumps(datos_nuevos)))
            
            conn.commit()
            print(f"✅ Empleado ID {empleado_id} actualizado exitosamente")
            print(f"📝 Campos modificados: {list(datos_nuevos.keys())}")
            return True
            
    except sqlite3.IntegrityError as e:
        print(f"❌ Error de integridad al actualizar: {e}")
        return False
    except Exception as e:
        print(f"❌ Error inesperado: {e}")
        return False

def actualizar_salarios_departamento(departamento: str, porcentaje_aumento: float) -> Dict[str, Any]:
    """Actualiza salarios de todo un departamento"""
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Obtener empleados del departamento
            cursor.execute("""
                SELECT id, nombre, apellido, salario 
                FROM empleados 
                WHERE departamento = ? AND activo = 1
            """, (departamento,))
            
            empleados = cursor.fetchall()
            
            if not empleados:
                print(f"❌ No se encontraron empleados en departamento: {departamento}")
                return {'success': False, 'empleados_actualizados': 0}
            
            empleados_actualizados = 0
            total_aumento = 0
            
            for empleado in empleados:
                emp_id, nombre, apellido, salario_actual = empleado
                nuevo_salario = salario_actual * (1 + porcentaje_aumento / 100)
                aumento = nuevo_salario - salario_actual
                
                # Actualizar salario individual
                cursor.execute("""
                    UPDATE empleados 
                    SET salario = ?, fecha_actualizacion = CURRENT_TIMESTAMP
                    WHERE id = ?
                """, (nuevo_salario, emp_id))
                
                # Auditoría
                cursor.execute("""
                    INSERT INTO auditoria_empleados (empleado_id, operacion, datos_nuevos)
                    VALUES (?, 'SALARY_UPDATE', ?)
                """, (emp_id, json.dumps({
                    'salario_anterior': salario_actual,
                    'salario_nuevo': nuevo_salario,
                    'aumento_porcentaje': porcentaje_aumento,
                    'aumento_valor': aumento,
                    'departamento': departamento
                })))
                
                empleados_actualizados += 1
                total_aumento += aumento
                
                print(f"💰 {nombre} {apellido}: ${salario_actual:,.2f} → ${nuevo_salario:,.2f} (+${aumento:,.2f})")
            
            conn.commit()
            
            resultado = {
                'success': True,
                'departamento': departamento,
                'empleados_actualizados': empleados_actualizados,
                'porcentaje_aumento': porcentaje_aumento,
                'total_aumento': total_aumento,
                'promedio_aumento': total_aumento / empleados_actualizados if empleados_actualizados > 0 else 0
            }
            
            print(f"✅ Actualización masiva completada:")
            print(f"   📊 Departamento: {departamento}")
            print(f"   👥 Empleados actualizados: {empleados_actualizados}")
            print(f"   📈 Aumento total: ${total_aumento:,.2f}")
            print(f"   📊 Aumento promedio: ${resultado['promedio_aumento']:,.2f}")
            
            return resultado
            
    except Exception as e:
        print(f"❌ Error en actualización masiva: {e}")
        return {'success': False, 'error': str(e)}

# 🧪 PRUEBAS DE ACTUALIZACIÓN
print("🧪 PROBANDO OPERACIONES UPDATE")
print("=" * 50)

# Actualización individual
if id_carlos:
    print(f"📝 ACTUALIZANDO EMPLEADO ID {id_carlos}:")
    
    # Mostrar datos actuales
    empleado_actual = obtener_empleado_por_id(id_carlos)
    if empleado_actual:
        print(f"   📋 Datos actuales:")
        print(f"      Teléfono: {empleado_actual['telefono']}")
        print(f"      Salario: ${empleado_actual['salario']:,.2f}")
        print(f"      Experiencia: {empleado_actual['años_experiencia']} años")
    
    # Actualizar datos
    nuevos_datos = {
        'telefono': '555-9999',
        'salario': 78000.00,
        'años_experiencia': 9
    }
    
    success = actualizar_empleado(id_carlos, nuevos_datos)
    
    if success:
        # Mostrar datos actualizados
        empleado_actualizado = obtener_empleado_por_id(id_carlos)
        print(f"   📋 Datos actualizados:")
        print(f"      Teléfono: {empleado_actualizado['telefono']}")
        print(f"      Salario: ${empleado_actualizado['salario']:,.2f}")
        print(f"      Experiencia: {empleado_actualizado['años_experiencia']} años")

# Actualización masiva por departamento
print(f"\n💼 APLICANDO AUMENTO DEL 5% AL DEPARTAMENTO DE AUTOMATIZACIÓN:")
resultado_aumento = actualizar_salarios_departamento('Automatización', 5.0)

# Verificar cambios
print(f"\n📊 VERIFICACIÓN POST-ACTUALIZACIÓN:")
empleados_auto_actualizados = buscar_empleados({'departamento': 'Automatización'})
for emp in empleados_auto_actualizados:
    print(f"   • {emp['nombre']} {emp['apellido']}: ${emp['salario']:,.2f}")

# Consultar auditoría
print(f"\n📋 ÚLTIMAS OPERACIONES DE AUDITORÍA:")
with obtener_conexion_segura('empresa_industrial.db') as conn:
    cursor = conn.cursor()
    cursor.execute("""
        SELECT a.*, e.nombre, e.apellido 
        FROM auditoria_empleados a
        LEFT JOIN empleados e ON a.empleado_id = e.id
        ORDER BY a.timestamp DESC 
        LIMIT 5
    """)
    
    auditoria = cursor.fetchall()
    for registro in auditoria:
        timestamp = registro['timestamp']
        operacion = registro['operacion']
        empleado = f"{registro['nombre']} {registro['apellido']}" if registro['nombre'] else "N/A"
        print(f"   📝 {timestamp}: {operacion} - {empleado}")

print("\n🧠 CONCEPTOS CLAVE UPDATE:")
print("✅ Auditoría completa de cambios")
print("✅ Validación de campos protegidos")  
print("✅ Actualización masiva con transacciones")
print("✅ Queries dinámicos para flexibilidad")
print("✅ Manejo robusto de errores")

### 🗑️ DELETE - Eliminación Segura

Implementaremos eliminación lógica (soft delete) y física, siempre con auditoría.

In [ ]:
# 🗑️ OPERACIONES DELETE (ELIMINACIÓN)
def eliminar_empleado_logico(empleado_id: int, motivo: str = "No especificado") -> bool:
    """Eliminación lógica (soft delete) - Marca como inactivo"""
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Verificar que el empleado existe y está activo
            cursor.execute("SELECT * FROM empleados WHERE id = ? AND activo = 1", (empleado_id,))
            empleado = cursor.fetchone()
            
            if not empleado:
                print(f"❌ Empleado ID {empleado_id} no encontrado o ya inactivo")
                return False
            
            datos_empleado = dict(empleado)
            
            # Marcar como inactivo
            cursor.execute("""
                UPDATE empleados 
                SET activo = 0, fecha_actualizacion = CURRENT_TIMESTAMP
                WHERE id = ?
            """, (empleado_id,))
            
            # Registrar auditoría
            cursor.execute("""
                INSERT INTO auditoria_empleados (empleado_id, operacion, datos_anteriores, datos_nuevos)
                VALUES (?, 'SOFT_DELETE', ?, ?)
            """, (empleado_id, json.dumps(datos_empleado), json.dumps({'motivo': motivo})))
            
            conn.commit()
            print(f"✅ Empleado ID {empleado_id} marcado como inactivo")
            print(f"📝 Motivo: {motivo}")
            print(f"👤 Empleado: {datos_empleado['nombre']} {datos_empleado['apellido']}")
            return True
            
    except Exception as e:
        print(f"❌ Error en eliminación lógica: {e}")
        return False

def restaurar_empleado(empleado_id: int) -> bool:
    """Restaura un empleado marcado como inactivo"""
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Verificar que el empleado existe y está inactivo
            cursor.execute("SELECT * FROM empleados WHERE id = ? AND activo = 0", (empleado_id,))
            empleado = cursor.fetchone()
            
            if not empleado:
                print(f"❌ Empleado ID {empleado_id} no encontrado o ya activo")
                return False
            
            datos_empleado = dict(empleado)
            
            # Reactivar empleado
            cursor.execute("""
                UPDATE empleados 
                SET activo = 1, fecha_actualizacion = CURRENT_TIMESTAMP
                WHERE id = ?
            """, (empleado_id,))
            
            # Registrar auditoría
            cursor.execute("""
                INSERT INTO auditoria_empleados (empleado_id, operacion, datos_nuevos)
                VALUES (?, 'RESTORE', ?)
            """, (empleado_id, json.dumps({'restaurado': True})))
            
            conn.commit()
            print(f"✅ Empleado ID {empleado_id} restaurado exitosamente")
            print(f"👤 Empleado: {datos_empleado['nombre']} {datos_empleado['apellido']}")
            return True
            
    except Exception as e:
        print(f"❌ Error en restauración: {e}")
        return False

def eliminar_empleado_fisico(empleado_id: int, confirmacion: bool = False) -> bool:
    """Eliminación física - CUIDADO: No se puede deshacer"""
    
    if not confirmacion:
        print("⚠️ ADVERTENCIA: Eliminación física requiere confirmación explícita")
        print("   Esta operación NO SE PUEDE DESHACER")
        print("   Use confirmacion=True para proceder")
        return False
    
    try:
        with obtener_conexion_segura('empresa_industrial.db') as conn:
            cursor = conn.cursor()
            
            # Obtener datos del empleado antes de eliminar
            cursor.execute("SELECT * FROM empleados WHERE id = ?", (empleado_id,))
            empleado = cursor.fetchone()
            
            if not empleado:
                print(f"❌ Empleado ID {empleado_id} no encontrado")
                return False
            
            datos_empleado = dict(empleado)
            
            # Registrar auditoría ANTES de eliminar
            cursor.execute("""
                INSERT INTO auditoria_empleados (empleado_id, operacion, datos_anteriores)
                VALUES (?, 'HARD_DELETE', ?)
            """, (empleado_id, json.dumps(datos_empleado)))
            
            # Eliminar físicamente
            cursor.execute("DELETE FROM empleados WHERE id = ?", (empleado_id,))
            
            conn.commit()
            print(f"🚨 Empleado ID {empleado_id} ELIMINADO FÍSICAMENTE")
            print(f"👤 Empleado eliminado: {datos_empleado['nombre']} {datos_empleado['apellido']}")
            print("⚠️ Esta operación no se puede deshacer")
            return True
            
    except Exception as e:
        print(f"❌ Error en eliminación física: {e}")
        return False

def obtener_empleados_inactivos() -> List[Dict[str, Any]]:
    """Obtiene lista de empleados inactivos para gestión"""
    
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT e.*, a.timestamp as fecha_eliminacion, a.datos_nuevos as motivo_json
            FROM empleados e
            LEFT JOIN auditoria_empleados a ON e.id = a.empleado_id 
                AND a.operacion = 'SOFT_DELETE'
                AND a.timestamp = (
                    SELECT MAX(timestamp) 
                    FROM auditoria_empleados a2 
                    WHERE a2.empleado_id = e.id AND a2.operacion = 'SOFT_DELETE'
                )
            WHERE e.activo = 0
            ORDER BY a.timestamp DESC
        """)
        
        return [dict(row) for row in cursor.fetchall()]

# 🧪 PRUEBAS DE ELIMINACIÓN
print("🧪 PROBANDO OPERACIONES DELETE")
print("=" * 50)

# Crear empleado temporal para pruebas
empleado_temporal = {
    'nombre': 'Pedro',
    'apellido': 'Temporal',
    'email': 'pedro.temporal@test.com',
    'salario': 50000.00,
    'departamento': 'Pruebas',
    'fecha_ingreso': '2024-01-01',
    'años_experiencia': 2
}

id_temporal = crear_empleado(empleado_temporal)

if id_temporal:
    print(f"\\n📝 EMPLEADO TEMPORAL CREADO (ID: {id_temporal})")
    
    # Mostrar empleados activos antes de eliminación
    print(f"\\n👥 EMPLEADOS ACTIVOS ANTES:")
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM empleados WHERE activo = 1")
        activos_antes = cursor.fetchone()[0]
        print(f"   Total activos: {activos_antes}")
    
    # Eliminación lógica
    print(f"\\n🗑️ ELIMINACIÓN LÓGICA:")
    eliminar_empleado_logico(id_temporal, "Prueba de eliminación lógica")
    
    # Mostrar empleados activos después
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM empleados WHERE activo = 1")
        activos_despues = cursor.fetchone()[0]
        print(f"   Total activos después: {activos_despues}")
    
    # Mostrar empleados inactivos
    print(f"\\n👻 EMPLEADOS INACTIVOS:")
    inactivos = obtener_empleados_inactivos()
    for emp in inactivos:
        motivo_data = json.loads(emp['motivo_json']) if emp['motivo_json'] else {}
        motivo = motivo_data.get('motivo', 'No especificado')
        print(f"   • {emp['nombre']} {emp['apellido']} - Motivo: {motivo}")
    
    # Restauración
    print(f"\\n🔄 RESTAURANDO EMPLEADO:")
    restaurar_empleado(id_temporal)
    
    # Verificar restauración
    with obtener_conexion_segura('empresa_industrial.db') as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM empleados WHERE activo = 1")
        activos_final = cursor.fetchone()[0]
        print(f"   Total activos después de restauración: {activos_final}")
    
    # Eliminación física (para limpieza)
    print(f"\\n🚨 ELIMINACIÓN FÍSICA (PRUEBA):")
    print("   Nota: En producción, esta operación sería muy restringida")
    eliminar_empleado_fisico(id_temporal, confirmacion=True)

print(f"\\n📊 RESUMEN FINAL DE EMPLEADOS:")
stats_finales = obtener_estadisticas_empleados()
print(f"   👥 Total empleados activos: {stats_finales['generales']['total_empleados']}")

print("\\n🧠 CONCEPTOS CLAVE DELETE:")
print("✅ Soft Delete: Preserva datos con flag inactivo")
print("✅ Hard Delete: Eliminación física irreversible")
print("✅ Auditoría completa de eliminaciones")
print("✅ Función de restauración para soft deletes")
print("✅ Gestión de empleados inactivos")

## 🏗️ 3. PATRONES DE DISEÑO PROFESIONALES

### 🎯 DAO (Data Access Object) Pattern

El patrón DAO encapsula todas las operaciones de base de datos en una clase organizada, facilitando el mantenimiento y reutilización del código.

In [ ]:
# 🏗️ CLASE DAO - DATA ACCESS OBJECT
class EmpleadoDAO:
    """Data Access Object para empleados - Patrón profesional"""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        self._inicializar_tabla()
    
    def _inicializar_tabla(self):
        """Inicializa las tablas necesarias"""
        # Ya las tenemos creadas, pero podríamos verificar/crear aquí
        pass
    
    # === OPERACIONES CRUD ===
    
    def crear(self, empleado: Dict[str, Any]) -> Optional[int]:
        """Crea un nuevo empleado"""
        return crear_empleado(empleado)
    
    def obtener_por_id(self, empleado_id: int) -> Optional[Dict[str, Any]]:
        """Obtiene empleado por ID"""
        return obtener_empleado_por_id(empleado_id)
    
    def buscar(self, filtros: Dict[str, Any] = None) -> List[Dict[str, Any]]:
        """Busca empleados con filtros"""
        return buscar_empleados(filtros)
    
    def actualizar(self, empleado_id: int, datos: Dict[str, Any]) -> bool:
        """Actualiza un empleado"""
        return actualizar_empleado(empleado_id, datos)
    
    def eliminar_logico(self, empleado_id: int, motivo: str = "") -> bool:
        """Eliminación lógica"""
        return eliminar_empleado_logico(empleado_id, motivo)
    
    def restaurar(self, empleado_id: int) -> bool:
        """Restaura empleado inactivo"""
        return restaurar_empleado(empleado_id)
    
    # === MÉTODOS DE NEGOCIO ESPECÍFICOS ===
    
    def obtener_por_departamento(self, departamento: str) -> List[Dict[str, Any]]:
        """Obtiene empleados de un departamento específico"""
        return self.buscar({'departamento': departamento})
    
    def obtener_por_rango_salarial(self, min_salario: float, max_salario: float) -> List[Dict[str, Any]]:
        """Obtiene empleados en rango salarial"""
        return self.buscar({'salario': {'min': min_salario, 'max': max_salario}})
    
    def contar_por_departamento(self) -> Dict[str, int]:
        """Cuenta empleados por departamento"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT departamento, COUNT(*) as total
                FROM empleados 
                WHERE activo = 1
                GROUP BY departamento
                ORDER BY total DESC
            """)
            
            return {row['departamento']: row['total'] for row in cursor.fetchall()}
    
    def obtener_top_salarios(self, limite: int = 5) -> List[Dict[str, Any]]:
        """Obtiene empleados con mejores salarios"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT * FROM empleados 
                WHERE activo = 1
                ORDER BY salario DESC 
                LIMIT ?
            """, (limite,))
            
            return [dict(row) for row in cursor.fetchall()]
    
    def obtener_antiguedad_promedio_departamento(self) -> Dict[str, float]:
        """Calcula antigüedad promedio por departamento"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT 
                    departamento,
                    ROUND(AVG(julianday('now') - julianday(fecha_ingreso)) / 365.25, 1) as años_promedio
                FROM empleados 
                WHERE activo = 1
                GROUP BY departamento
                ORDER BY años_promedio DESC
            """)
            
            return {row['departamento']: row['años_promedio'] for row in cursor.fetchall()}
    
    def generar_reporte_completo(self) -> Dict[str, Any]:
        """Genera reporte completo del sistema"""
        return {
            'estadisticas_generales': obtener_estadisticas_empleados(),
            'conteo_por_departamento': self.contar_por_departamento(),
            'top_salarios': self.obtener_top_salarios(3),
            'antiguedad_departamentos': self.obtener_antiguedad_promedio_departamento(),
            'timestamp': datetime.now().isoformat()
        }

class SensorDAO:
    """DAO para sensores industriales"""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        self._inicializar_tablas()
    
    def _inicializar_tablas(self):
        """Crea tablas de sensores si no existen"""
        with obtener_conexion_segura(self.db_path) as conn:
            conn.executescript("""
                CREATE TABLE IF NOT EXISTS sensores (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    nombre TEXT NOT NULL,
                    tipo TEXT NOT NULL,
                    ubicacion TEXT,
                    unidad_medida TEXT,
                    valor_min REAL,
                    valor_max REAL,
                    activo BOOLEAN DEFAULT 1,
                    fecha_creacion DATETIME DEFAULT CURRENT_TIMESTAMP
                );
                
                CREATE TABLE IF NOT EXISTS lecturas_sensores (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    sensor_id INTEGER,
                    valor REAL NOT NULL,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    estado TEXT DEFAULT 'NORMAL',
                    FOREIGN KEY (sensor_id) REFERENCES sensores(id)
                );
                
                CREATE INDEX IF NOT EXISTS idx_lecturas_sensor_time 
                ON lecturas_sensores(sensor_id, timestamp);
            """)
            conn.commit()
    
    def crear_sensor(self, sensor_data: Dict[str, Any]) -> int:
        """Crea un nuevo sensor"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                INSERT INTO sensores (nombre, tipo, ubicacion, unidad_medida, valor_min, valor_max)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                sensor_data['nombre'], sensor_data['tipo'],
                sensor_data['ubicacion'], sensor_data['unidad_medida'],
                sensor_data['valor_min'], sensor_data['valor_max']
            ))
            conn.commit()
            return cursor.lastrowid
    
    def registrar_lectura(self, sensor_id: int, valor: float) -> bool:
        """Registra una lectura del sensor"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            
            # Obtener límites del sensor
            cursor.execute("SELECT valor_min, valor_max FROM sensores WHERE id = ?", (sensor_id,))
            sensor = cursor.fetchone()
            
            if not sensor:
                return False
            
            valor_min, valor_max = sensor
            
            # Determinar estado
            if valor < valor_min:
                estado = 'BAJO'
            elif valor > valor_max:
                estado = 'ALTO'
            else:
                estado = 'NORMAL'
            
            # Registrar lectura
            cursor.execute("""
                INSERT INTO lecturas_sensores (sensor_id, valor, estado)
                VALUES (?, ?, ?)
            """, (sensor_id, valor, estado))
            
            conn.commit()
            return True
    
    def obtener_lecturas_recientes(self, sensor_id: int, limite: int = 10) -> List[Dict[str, Any]]:
        """Obtiene las lecturas más recientes de un sensor"""
        with obtener_conexion_segura(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute("""
                SELECT * FROM lecturas_sensores 
                WHERE sensor_id = ?
                ORDER BY timestamp DESC 
                LIMIT ?
            """, (sensor_id, limite))
            
            return [dict(row) for row in cursor.fetchall()]

# 🧪 DEMO DEL PATRÓN DAO
print("🧪 DEMOSTRANDO PATRÓN DAO")
print("=" * 50)

# Crear instancia del DAO
empleado_dao = EmpleadoDAO('empresa_industrial.db')
sensor_dao = SensorDAO('empresa_industrial.db')

print("✅ DAOs inicializados")

# Usar métodos de negocio específicos
print(f"\\n📊 CONTEO POR DEPARTAMENTO:")
conteo_depto = empleado_dao.contar_por_departamento()
for depto, cantidad in conteo_depto.items():
    print(f"   🏢 {depto}: {cantidad} empleados")

print(f"\\n🏆 TOP 3 SALARIOS:")
top_salarios = empleado_dao.obtener_top_salarios(3)
for i, emp in enumerate(top_salarios, 1):
    print(f"   #{i} {emp['nombre']} {emp['apellido']}: ${emp['salario']:,.2f}")

print(f"\\n⏰ ANTIGÜEDAD PROMEDIO POR DEPARTAMENTO:")
antiguedad = empleado_dao.obtener_antiguedad_promedio_departamento()
for depto, años in antiguedad.items():
    print(f"   🏢 {depto}: {años} años promedio")

# Demo de sensores
print(f"\\n🔧 CREANDO SENSOR DE TEMPERATURA:")
sensor_temp = {
    'nombre': 'Temperatura Reactor Principal',
    'tipo': 'Temperatura',
    'ubicacion': 'Planta - Reactor 1',
    'unidad_medida': '°C',
    'valor_min': 20.0,
    'valor_max': 80.0
}

sensor_id = sensor_dao.crear_sensor(sensor_temp)
print(f"   ✅ Sensor creado con ID: {sensor_id}")

# Simular algunas lecturas
print(f"\\n📊 SIMULANDO LECTURAS:")
lecturas_demo = [75.5, 82.1, 78.3, 85.0, 76.8]  # Algunos valores fuera de rango
for valor in lecturas_demo:
    sensor_dao.registrar_lectura(sensor_id, valor)
    print(f"   📈 Lectura registrada: {valor}°C")

# Obtener lecturas recientes
print(f"\\n📋 LECTURAS RECIENTES:")
lecturas = sensor_dao.obtener_lecturas_recientes(sensor_id, 5)
for lectura in lecturas:
    timestamp = lectura['timestamp']
    valor = lectura['valor']
    estado = lectura['estado']
    icono = "🔴" if estado == 'ALTO' else "🔵" if estado == 'BAJO' else "🟢"
    print(f"   {icono} {timestamp}: {valor}°C ({estado})")

print("\\n🧠 VENTAJAS DEL PATRÓN DAO:")
print("✅ Encapsulación de operaciones de BD")
print("✅ Métodos de negocio específicos")
print("✅ Reutilización de código")
print("✅ Fácil testing y mantenimiento")
print("✅ Separación de responsabilidades")

## 📊 4. PANDAS + SQL - ANÁLISIS HÍBRIDO

### 🔗 La Potencia de Combinar SQL y Pandas

Pandas ofrece funciones específicas para trabajar con SQL, permitiendo aprovechar lo mejor de ambos mundos: la eficiencia de SQL y las capacidades analíticas de Pandas.

In [ ]:
# 📊 ANÁLISIS HÍBRIDO PANDAS + SQL
class AnalisisIndustrial:
    """Clase para análisis avanzado de datos industriales"""
    
    def __init__(self, db_path: str):
        self.db_path = db_path
    
    def obtener_conexion_pandas(self):
        """Conexión optimizada para pandas"""
        return sqlite3.connect(self.db_path)
    
    def empleados_to_dataframe(self) -> pd.DataFrame:
        """Convierte empleados a DataFrame para análisis"""
        query = """
            SELECT 
                nombre || ' ' || apellido as nombre_completo,
                email, salario, departamento, 
                fecha_ingreso, años_experiencia,
                julianday('now') - julianday(fecha_ingreso) as dias_empresa
            FROM empleados 
            WHERE activo = 1
        """
        
        with self.obtener_conexion_pandas() as conn:
            df = pd.read_sql_query(query, conn)
            
            # Conversiones de tipos
            df['fecha_ingreso'] = pd.to_datetime(df['fecha_ingreso'])
            df['años_empresa'] = df['dias_empresa'] / 365.25
            
            return df
    
    def analisis_salarial_avanzado(self) -> Dict[str, Any]:
        """Análisis salarial usando pandas"""
        df = self.empleados_to_dataframe()
        
        # Estadísticas descriptivas completas
        stats_salario = df['salario'].describe()
        
        # Análisis por departamento
        depto_analysis = df.groupby('departamento').agg({
            'salario': ['mean', 'median', 'std', 'min', 'max', 'count'],
            'años_experiencia': 'mean',
            'años_empresa': 'mean'
        }).round(2)
        
        # Correlaciones
        correlaciones = df[['salario', 'años_experiencia', 'años_empresa']].corr()
        
        # Detección de outliers (usando IQR)
        Q1 = df['salario'].quantile(0.25)
        Q3 = df['salario'].quantile(0.75)
        IQR = Q3 - Q1
        outliers_inferior = Q1 - 1.5 * IQR
        outliers_superior = Q3 + 1.5 * IQR
        
        outliers = df[(df['salario'] < outliers_inferior) | (df['salario'] > outliers_superior)]
        
        return {
            'estadisticas_descriptivas': stats_salario.to_dict(),
            'analisis_departamental': depto_analysis.to_dict(),
            'correlaciones': correlaciones.to_dict(),
            'outliers_salariales': outliers[['nombre_completo', 'departamento', 'salario']].to_dict('records'),
            'resumen': {
                'total_empleados': len(df),
                'salario_promedio': df['salario'].mean(),
                'brecha_salarial': df['salario'].max() - df['salario'].min(),
                'coef_variacion': (df['salario'].std() / df['salario'].mean()) * 100
            }
        }
    
    def tendencias_temporales(self) -> pd.DataFrame:
        """Análisis de tendencias temporales de contratación"""
        df = self.empleados_to_dataframe()
        
        # Crear series temporal de contrataciones
        df['año_ingreso'] = df['fecha_ingreso'].dt.year
        df['mes_ingreso'] = df['fecha_ingreso'].dt.month
        
        # Análisis mensual
        contrataciones_mes = df.groupby(['año_ingreso', 'mes_ingreso']).agg({
            'nombre_completo': 'count',
            'salario': 'mean'
        }).rename(columns={
            'nombre_completo': 'contrataciones',
            'salario': 'salario_promedio_ingreso'
        })
        
        # Análisis anual
        contrataciones_año = df.groupby('año_ingreso').agg({
            'nombre_completo': 'count',
            'salario': ['mean', 'median'],
            'años_experiencia': 'mean'
        })
        
        return {
            'mensual': contrataciones_mes.reset_index(),
            'anual': contrataciones_año.reset_index()
        }
    
    def generar_reporte_excel_avanzado(self, archivo: str):
        """Genera reporte Excel con múltiples hojas y análisis"""
        
        # Obtener datos
        df_empleados = self.empleados_to_dataframe()
        analisis_salarial = self.analisis_salarial_avanzado()
        tendencias = self.tendencias_temporales()
        
        # Crear archivo Excel con múltiples hojas
        with pd.ExcelWriter(archivo, engine='openpyxl') as writer:
            
            # Hoja 1: Datos crudos
            df_empleados.to_excel(writer, sheet_name='Datos Empleados', index=False)
            
            # Hoja 2: Análisis departamental
            depto_df = pd.DataFrame(analisis_salarial['analisis_departamental']).T
            depto_df.to_excel(writer, sheet_name='Análisis Departamental')
            
            # Hoja 3: Tendencias anuales
            tendencias['anual'].to_excel(writer, sheet_name='Tendencias Anuales', index=False)
            
            # Hoja 4: Outliers
            outliers_df = pd.DataFrame(analisis_salarial['outliers_salariales'])
            if not outliers_df.empty:
                outliers_df.to_excel(writer, sheet_name='Outliers Salariales', index=False)
            
            # Hoja 5: Resumen ejecutivo
            resumen_df = pd.DataFrame([analisis_salarial['resumen']])
            resumen_df.to_excel(writer, sheet_name='Resumen Ejecutivo', index=False)
        
        print(f"✅ Reporte Excel generado: {archivo}")
        return archivo
    
    def dataframe_to_sql_demo(self):
        """Demuestra cómo insertar DataFrame en SQL"""
        
        # Crear DataFrame de ejemplo (nuevos empleados)
        nuevos_empleados = pd.DataFrame([
            {
                'nombre': 'Sofia', 'apellido': 'Ramirez', 
                'email': 'sofia.ramirez@industrial.com',
                'salario': 71000, 'departamento': 'Automatización',
                'fecha_ingreso': '2024-06-01', 'años_experiencia': 4
            },
            {
                'nombre': 'Miguel', 'apellido': 'Torres',
                'email': 'miguel.torres@industrial.com', 
                'salario': 63000, 'departamento': 'Calidad',
                'fecha_ingreso': '2024-06-15', 'años_experiencia': 6
            }
        ])
        
        print("📊 INSERTANDO DATAFRAME EN SQL:")
        print(nuevos_empleados)
        
        # Insertar en base de datos
        with self.obtener_conexion_pandas() as conn:
            # if_exists='append' agrega a tabla existente
            nuevos_empleados.to_sql('empleados', conn, if_exists='append', index=False)
        
        print("✅ DataFrame insertado en tabla 'empleados'")
        return len(nuevos_empleados)

# 🧪 DEMO DE ANÁLISIS HÍBRIDO
print("🧪 DEMOSTRANDO ANÁLISIS HÍBRIDO PANDAS + SQL")
print("=" * 60)

# Crear instancia del analizador
analizador = AnalisisIndustrial('empresa_industrial.db')

# 1. Convertir a DataFrame
print("\\n📊 CONVIRTIENDO EMPLEADOS A DATAFRAME:")
df_empleados = analizador.empleados_to_dataframe()
print(f"   📈 Shape: {df_empleados.shape}")
print(f"   📋 Columnas: {list(df_empleados.columns)}")

# Mostrar primeras filas
print("\\n👀 PRIMERAS 3 FILAS:")
print(df_empleados.head(3).to_string())

# 2. Análisis salarial avanzado
print("\\n💰 ANÁLISIS SALARIAL AVANZADO:")
analisis = analizador.analisis_salarial_avanzado()

print(f"\\n📊 ESTADÍSTICAS DESCRIPTIVAS:")
stats = analisis['estadisticas_descriptivas']
print(f"   💰 Promedio: ${stats['mean']:,.2f}")
print(f"   📈 Mediana: ${stats['50%']:,.2f}")
print(f"   📊 Desviación estándar: ${stats['std']:,.2f}")
print(f"   📉 Mínimo: ${stats['min']:,.2f}")
print(f"   📈 Máximo: ${stats['max']:,.2f}")

print(f"\\n🎯 RESUMEN EJECUTIVO:")
resumen = analisis['resumen']
print(f"   👥 Total empleados: {resumen['total_empleados']}")
print(f"   💰 Salario promedio: ${resumen['salario_promedio']:,.2f}")
print(f"   📊 Brecha salarial: ${resumen['brecha_salarial']:,.2f}")
print(f"   📈 Coeficiente variación: {resumen['coef_variacion']:.1f}%")

# Mostrar outliers si existen
outliers = analisis['outliers_salariales']
if outliers:
    print(f"\\n🔍 OUTLIERS SALARIALES:")
    for outlier in outliers:
        print(f"   • {outlier['nombre_completo']} ({outlier['departamento']}): ${outlier['salario']:,.2f}")
else:
    print("\\n✅ No se detectaron outliers salariales")

# 3. Correlaciones
print(f"\\n🔗 CORRELACIONES:")
correlaciones = analisis['correlaciones']
print(f"   📊 Salario vs Experiencia: {correlaciones['salario']['años_experiencia']:.3f}")
print(f"   📊 Salario vs Años Empresa: {correlaciones['salario']['años_empresa']:.3f}")
print(f"   📊 Experiencia vs Años Empresa: {correlaciones['años_experiencia']['años_empresa']:.3f}")

# 4. Demo de DataFrame to SQL
print(f"\\n💾 DEMO: DATAFRAME TO SQL")
empleados_agregados = analizador.dataframe_to_sql_demo()
print(f"   ✅ {empleados_agregados} empleados agregados desde DataFrame")

# 5. Generar reporte Excel
print(f"\\n📄 GENERANDO REPORTE EXCEL AVANZADO:")
archivo_excel = "reporte_empleados_avanzado.xlsx"
analizador.generar_reporte_excel_avanzado(archivo_excel)

# Verificar total actualizado
with obtener_conexion_segura('empresa_industrial.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM empleados WHERE activo = 1")
    total_final = cursor.fetchone()[0]
    print(f"\\n📈 Total empleados después de inserciones: {total_final}")

print("\\n🧠 VENTAJAS DEL ANÁLISIS HÍBRIDO:")
print("✅ SQL para consultas eficientes")
print("✅ Pandas para análisis estadístico avanzado")
print("✅ Exportación directa a Excel/CSV")
print("✅ Detección automática de outliers")
print("✅ Cálculo de correlaciones")
print("✅ Inserción masiva desde DataFrames")

## 🎓 CONSOLIDACIÓN Y PRÓXIMOS PASOS

### ✅ CONCEPTOS DOMINADOS EN ESTE MÓDULO

**🔌 Integración Python-SQLite:**
- Context managers para conexiones seguras
- Manejo robusto de errores y transacciones
- Configuración optimizada de conexiones

**📊 Operaciones CRUD Avanzadas:**
- Inserción con validación y auditoría
- Consultas dinámicas con filtros flexibles
- Actualización masiva con transacciones
- Eliminación lógica vs física

**🏗️ Patrones de Diseño:**
- DAO (Data Access Object) pattern
- Encapsulación de lógica de negocio
- Separación de responsabilidades
- Métodos especializados por dominio

**📈 Análisis Híbrido Pandas-SQL:**
- Conversión SQL → DataFrame
- Estadísticas descriptivas avanzadas
- Detección de outliers automática
- Exportación a múltiples formatos
- Inserción masiva desde DataFrames

### 🎯 APLICACIONES INDUSTRIALES DESARROLLADAS

1. **Sistema de Gestión de Empleados**
   - CRUD completo con auditoría
   - Búsquedas avanzadas y filtros
   - Análisis estadístico de recursos humanos

2. **Sistema de Monitoreo de Sensores**
   - Registro automático de lecturas
   - Detección de valores fuera de rango
   - Historial temporal de mediciones

3. **Generador de Reportes Automatizado**
   - Exportación a Excel multi-hoja
   - Análisis de tendencias temporales
   - Métricas de negocio automatizadas

---

### 🧠 EVALUACIÓN FINAL - VERIFIQUE SU DOMINIO

**Marque ✅ cada concepto que domina completamente:**

**🔌 CONEXIONES Y MANEJO:**
- [ ] Uso de context managers para SQLite
- [ ] Manejo adecuado de errores de BD
- [ ] Configuración de row_factory
- [ ] Transacciones seguras

**📊 OPERACIONES CRUD:**
- [ ] Inserción con validación de integridad
- [ ] Consultas dinámicas con filtros
- [ ] Actualización con auditoría
- [ ] Eliminación lógica vs física

**🏗️ PATRONES PROFESIONALES:**
- [ ] Implementación del patrón DAO
- [ ] Encapsulación de lógica de negocio
- [ ] Métodos especializados por dominio
- [ ] Separación de responsabilidades

**📈 ANÁLISIS AVANZADO:**
- [ ] Integración Pandas + SQL
- [ ] Análisis estadístico de datos
- [ ] Exportación a múltiples formatos
- [ ] Detección automática de outliers

**💼 APLICACIONES INDUSTRIALES:**
- [ ] Sistema de gestión de empleados
- [ ] Monitoreo de sensores IoT
- [ ] Generación automática de reportes
- [ ] Dashboard de métricas empresariales

---

### 🚀 ROADMAP - PRÓXIMOS SUBMÓDULOS

**🎯 SUBMÓDULO 3.3: CONSULTAS AVANZADAS Y OPTIMIZACIÓN**
- Views y stored procedures
- Triggers para automatización
- Optimización de consultas complejas
- Análisis de performance con EXPLAIN

**🎯 SUBMÓDULO 3.4: ORM CON SQLALCHEMY**
- Models y relaciones complejas
- Migrations automáticas
- Query builder avanzado
- Integración con frameworks web

**🎯 SUBMÓDULO 3.5: APIS REST CON BASES DE DATOS**
- FastAPI + SQLAlchemy
- Endpoints CRUD automatizados
- Autenticación y autorización
- Deploy en producción

**🎯 PROYECTO FINAL FASE 3:**
- Sistema completo de monitoreo industrial
- Dashboard web en tiempo real
- APIs REST para integración
- Base de datos optimizada para producción

---

### ✅ CONFIRMACIÓN DE CONSOLIDACIÓN

**🔥 Para marcar este submódulo como CONSOLIDADO:**

1. ✅ Ejecute exitosamente todos los ejemplos del notebook
2. ✅ Implemente su propio DAO para un dominio diferente
3. ✅ Cree un análisis híbrido Pandas-SQL personalizado
4. ✅ Genere un reporte automatizado de sus propios datos
5. ✅ Marque todos los ítems de la evaluación

**📝 Escriba "CONSOLIDADO 3.2" cuando domine completamente:** _________

---

**🎉 ¡EXCELENTE PROGRESO!**

Has dominado la integración Python-SQLite, base fundamental para automatización industrial. Ahora estás listo para consultas avanzadas y optimización en el siguiente submódulo.

**💪 TU NIVEL ACTUAL:**
- ✅ Fundamentos SQL (Módulo 3.1)
- ✅ Integración Python-SQLite (Módulo 3.2)
- 🎯 **PRÓXIMO:** Consultas Avanzadas y Optimización (Módulo 3.3)

In [ ]:
# 🔧 VERIFICACIÓN Y CONFIGURACIÓN DEL ENTORNO
import sys
import sqlite3
import pandas as pd
from datetime import datetime, date
import json
import csv
from pathlib import Path
import os

print("🔍 VERIFICANDO ENTORNO DE DESARROLLO:")
print("=" * 50)
print(f"🐍 Python Version: {sys.version}")
print(f"📊 Pandas Version: {pd.__version__}")
print(f"🗄️ SQLite Version: {sqlite3.sqlite_version}")
print(f"💾 SQLite3 Module: {sqlite3.version}")
print(f"📁 Working Directory: {os.getcwd()}")
print("=" * 50)
print("✅ ¡Entorno listo para trabajar con Python + SQLite!")

## 📚 1. CONEXIÓN PYTHON-SQLITE

### 🤔 ¿Por qué SQLite + Python?

**SQLite** es la base de datos perfecta para aplicaciones industriales porque:
- ✅ **Sin servidor**: No requiere instalación adicional
- ✅ **Embebida**: Se integra directamente en tu aplicación
- ✅ **Rápida**: Ideal para aplicaciones de tiempo real
- ✅ **Confiable**: Transacciones ACID completas
- ✅ **Portable**: Un solo archivo .db

### 🔌 Conexión Básica

El módulo `sqlite3` viene incluido con Python, ¡no necesitas instalar nada!

In [ ]:
# 🔌 CONEXIÓN BÁSICA A SQLITE
import sqlite3
from datetime import datetime

# Conectar a base de datos (se crea automáticamente si no existe)
conn = sqlite3.connect('sistema_industrial.db')
cursor = conn.cursor()

print("✅ Conexión establecida exitosamente")
print(f"📁 Base de datos creada: {os.path.abspath('sistema_industrial.db')}")

# Verificar conexión ejecutando una consulta simple
cursor.execute("SELECT sqlite_version()")
version = cursor.fetchone()
print(f"🗄️ SQLite versión en uso: {version[0]}")

# IMPORTANTE: Siempre cerrar conexiones
conn.close()
print("🔒 Conexión cerrada correctamente")

### 🔐 Context Managers: La Forma Profesional

Los **context managers** garantizan que las conexiones se manejen correctamente, incluso si ocurre un error.

**🔥 VENTAJAS:**
- Cierre automático de conexiones
- Manejo de errores robusto
- Código más limpio y legible
- Rollback automático en errores

In [ ]:
# 🔐 CONTEXT MANAGER PERSONALIZADO
from contextlib import contextmanager

@contextmanager
def obtener_conexion_db(db_path: str = 'sistema_industrial.db'):
    """
    Context manager para manejo seguro de conexiones SQLite
    
    Args:
        db_path: Ruta a la base de datos
        
    Yields:
        sqlite3.Connection: Conexión activa a la base de datos
    """
    conn = None
    try:
        # Establecer conexión
        conn = sqlite3.connect(db_path)
        
        # Configurar para devolver resultados como diccionarios
        conn.row_factory = sqlite3.Row
        
        print(f"🔌 Conexión establecida: {db_path}")
        yield conn
        
    except sqlite3.Error as e:
        if conn:
            conn.rollback()
            print(f"❌ Error en base de datos: {e}")
            print("🔄 Rollback ejecutado")
        raise
        
    finally:
        if conn:
            conn.close()
            print("🔒 Conexión cerrada automáticamente")

# 🧪 PROBANDO EL CONTEXT MANAGER
print("🧪 DEMO: Context Manager en acción")
print("-" * 40)

with obtener_conexion_db() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT 'Context Manager funcionando!' as mensaje")
    resultado = cursor.fetchone()
    print(f"✅ Resultado: {resultado['mensaje']}")
    
print("🎯 ¡El context manager cerró la conexión automáticamente!")

## 🏗️ 2. CREACIÓN DE ESQUEMAS Y TABLAS

### 🎯 Diseño de Base de Datos Industrial

Vamos a crear un sistema completo para una **planta industrial** con:
- 🏭 **Sensores**: Dispositivos de monitoreo (temperatura, presión, caudal)
- 📊 **Lecturas**: Datos históricos de sensores en tiempo real
- ⚠️ **Alarmas**: Sistema de alertas automáticas
- 👥 **Operadores**: Personal de planta y turnos
- ⚙️ **Configuración**: Parámetros del sistema

In [ ]:
# 🏗️ CREACIÓN DEL ESQUEMA DE BASE DE DATOS INDUSTRIAL

def crear_esquema_industrial():
    """Crea el esquema completo para el sistema industrial"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # 📊 TABLA DE SENSORES
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS sensores (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nombre TEXT NOT NULL UNIQUE,
                tipo TEXT NOT NULL,
                ubicacion TEXT NOT NULL,
                unidad_medida TEXT NOT NULL,
                valor_min REAL,
                valor_max REAL,
                activo BOOLEAN DEFAULT 1,
                fecha_instalacion DATE DEFAULT CURRENT_DATE,
                ultima_calibracion DATE,
                intervalo_lectura INTEGER DEFAULT 60,
                descripcion TEXT
            );
        ''')
        
        # 📈 TABLA DE LECTURAS
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS lecturas (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                sensor_id INTEGER NOT NULL,
                valor REAL NOT NULL,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                estado TEXT DEFAULT 'NORMAL',
                calidad TEXT DEFAULT 'BUENA',
                operador_id INTEGER,
                observaciones TEXT,
                FOREIGN KEY (sensor_id) REFERENCES sensores(id),
                FOREIGN KEY (operador_id) REFERENCES operadores(id)
            );
        ''')
        
        # ⚠️ TABLA DE ALARMAS
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS alarmas (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                sensor_id INTEGER NOT NULL,
                tipo_alarma TEXT NOT NULL,
                severidad TEXT DEFAULT 'MEDIA',
                mensaje TEXT NOT NULL,
                valor_activacion REAL,
                timestamp_activacion DATETIME DEFAULT CURRENT_TIMESTAMP,
                timestamp_reconocimiento DATETIME,
                timestamp_resolucion DATETIME,
                operador_reconocimiento INTEGER,
                operador_resolucion INTEGER,
                estado TEXT DEFAULT 'ACTIVA',
                acciones_tomadas TEXT,
                FOREIGN KEY (sensor_id) REFERENCES sensores(id),
                FOREIGN KEY (operador_reconocimiento) REFERENCES operadores(id),
                FOREIGN KEY (operador_resolucion) REFERENCES operadores(id)
            );
        ''')
        
        # 👥 TABLA DE OPERADORES
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS operadores (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                nombre TEXT NOT NULL,
                apellido TEXT NOT NULL,
                usuario TEXT UNIQUE NOT NULL,
                email TEXT UNIQUE,
                turno TEXT,
                nivel_acceso TEXT DEFAULT 'OPERADOR',
                activo BOOLEAN DEFAULT 1,
                fecha_ingreso DATE DEFAULT CURRENT_DATE,
                ultimo_acceso DATETIME
            );
        ''')
        
        # 📊 CREAR ÍNDICES PARA OPTIMIZACIÓN
        indices = [
            "CREATE INDEX IF NOT EXISTS idx_lecturas_sensor_time ON lecturas(sensor_id, timestamp);",
            "CREATE INDEX IF NOT EXISTS idx_lecturas_timestamp ON lecturas(timestamp);",
            "CREATE INDEX IF NOT EXISTS idx_alarmas_estado ON alarmas(estado, timestamp_activacion);",
            "CREATE INDEX IF NOT EXISTS idx_sensores_activo ON sensores(activo);",
            "CREATE INDEX IF NOT EXISTS idx_operadores_usuario ON operadores(usuario);"
        ]
        
        for indice in indices:
            cursor.execute(indice)
        
        # 💾 CONFIRMAR TODOS LOS CAMBIOS
        conn.commit()
        
        print("✅ Esquema de base de datos industrial creado exitosamente")
        return True

# 🚀 EJECUTAR CREACIÓN DEL ESQUEMA
print("🏗️ CREANDO ESQUEMA DE BASE DE DATOS INDUSTRIAL")
print("=" * 55)

try:
    crear_esquema_industrial()
    
    # Verificar tablas creadas
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
        tablas = cursor.fetchall()
        
        print(f"\n📋 Tablas creadas ({len(tablas)}):")
        for tabla in tablas:
            print(f"   🗂️ {tabla['name']}")
            
        # Verificar índices
        cursor.execute("SELECT name FROM sqlite_master WHERE type='index' AND name LIKE 'idx_%';")
        indices = cursor.fetchall()
        
        print(f"\n⚡ Índices creados ({len(indices)}):")
        for indice in indices:
            print(f"   📊 {indice['name']}")
            
except Exception as e:
    print(f"❌ Error al crear esquema: {e}")

## 📝 3. OPERACIONES CRUD CON PYTHON

### 🔥 ¿Qué son las operaciones CRUD?

**CRUD** son las 4 operaciones fundamentales en bases de datos:
- **🟢 CREATE**: Insertar nuevos registros
- **🔵 READ**: Leer y consultar datos
- **🟡 UPDATE**: Actualizar registros existentes  
- **🔴 DELETE**: Eliminar registros

### 🟢 CREATE - Insertar Datos

Vamos a poblar nuestra base de datos con datos industriales realistas.

In [ ]:
# 🟢 CREATE - INSERTAR DATOS INDUSTRIALES

def insertar_datos_iniciales():
    """Inserta datos de ejemplo en el sistema industrial"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # 👥 INSERTAR OPERADORES
        operadores_data = [
            ('Juan', 'Pérez', 'jperez', 'juan.perez@planta.com', 'MAÑANA', 'SUPERVISOR'),
            ('María', 'González', 'mgonzalez', 'maria.gonzalez@planta.com', 'TARDE', 'OPERADOR'),
            ('Carlos', 'Rodríguez', 'crodriguez', 'carlos.rodriguez@planta.com', 'NOCHE', 'OPERADOR'),
            ('Ana', 'López', 'alopez', 'ana.lopez@planta.com', 'MAÑANA', 'TECNICO'),
            ('Pedro', 'Martín', 'pmartin', 'pedro.martin@planta.com', 'TARDE', 'SUPERVISOR')
        ]
        
        cursor.executemany('''
            INSERT INTO operadores (nombre, apellido, usuario, email, turno, nivel_acceso)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', operadores_data)
        
        print(f"✅ Insertados {len(operadores_data)} operadores")
        
        # 🏭 INSERTAR SENSORES
        sensores_data = [
            ('Temperatura Reactor 1', 'TEMPERATURA', 'Reactor Principal - Planta A', '°C', 20.0, 80.0, '2023-01-15', 60, 'Sensor de temperatura crítica del reactor principal'),
            ('Presión Línea Vapor', 'PRESION', 'Línea Principal - Sector B', 'Bar', 0.5, 15.0, '2023-02-10', 30, 'Monitoreo de presión en línea de vapor'),
            ('Caudal Agua Enfriamiento', 'CAUDAL', 'Sistema Enfriamiento - Planta A', 'L/min', 50.0, 200.0, '2023-01-20', 45, 'Control de flujo de agua de enfriamiento'),
            ('Nivel Tanque Principal', 'NIVEL', 'Tanque Almacenamiento - Sector C', '%', 10.0, 95.0, '2023-03-05', 120, 'Nivel de llenado del tanque principal'),
            ('Vibración Motor Bomba', 'VIBRACION', 'Sala Bombas - Planta A', 'mm/s', 0.1, 10.0, '2023-02-28', 60, 'Monitoreo de vibración del motor principal'),
            ('pH Proceso Químico', 'PH', 'Reactor Químico - Sector D', 'pH', 6.5, 8.5, '2023-01-10', 30, 'Control de pH en proceso químico'),
            ('Temperatura Ambiente', 'TEMPERATURA', 'Sala Control - Planta A', '°C', 18.0, 25.0, '2023-01-01', 300, 'Temperatura ambiente en sala de control'),
            ('Humedad Relativa', 'HUMEDAD', 'Almacén Materias Primas', '%', 30.0, 70.0, '2023-02-15', 180, 'Control de humedad en almacén')
        ]
        
        cursor.executemany('''
            INSERT INTO sensores (nombre, tipo, ubicacion, unidad_medida, valor_min, valor_max, 
                                fecha_instalacion, intervalo_lectura, descripcion)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', sensores_data)
        
        print(f"✅ Insertados {len(sensores_data)} sensores")
        
        # 📊 INSERTAR LECTURAS DE EJEMPLO (últimas 24 horas simuladas)
        import random
        from datetime import timedelta
        
        lecturas_generadas = 0
        base_time = datetime.now() - timedelta(hours=24)
        
        # Obtener IDs de sensores y operadores
        cursor.execute("SELECT id FROM sensores")
        sensor_ids = [row['id'] for row in cursor.fetchall()]
        
        cursor.execute("SELECT id FROM operadores")
        operador_ids = [row['id'] for row in cursor.fetchall()]
        
        # Generar lecturas para cada sensor
        for sensor_id in sensor_ids:
            # Obtener rangos del sensor
            cursor.execute("SELECT valor_min, valor_max, intervalo_lectura FROM sensores WHERE id = ?", (sensor_id,))
            sensor_info = cursor.fetchone()
            min_val, max_val, intervalo = sensor_info['valor_min'], sensor_info['valor_max'], sensor_info['intervalo_lectura']
            
            # Generar lecturas cada 'intervalo' minutos durante 24 horas
            current_time = base_time
            while current_time <= datetime.now():
                # Generar valor dentro del rango (90% normal, 10% fuera de rango)
                if random.random() < 0.9:
                    # Valor normal
                    valor = random.uniform(min_val + (max_val - min_val) * 0.1, 
                                         max_val - (max_val - min_val) * 0.1)
                    estado = 'NORMAL'
                else:
                    # Valor fuera de rango
                    if random.random() < 0.5:
                        valor = random.uniform(min_val - (max_val - min_val) * 0.1, min_val)
                        estado = 'BAJO'
                    else:
                        valor = random.uniform(max_val, max_val + (max_val - min_val) * 0.1)
                        estado = 'ALTO'
                
                # Insertar lectura
                cursor.execute('''
                    INSERT INTO lecturas (sensor_id, valor, timestamp, estado, operador_id)
                    VALUES (?, ?, ?, ?, ?)
                ''', (sensor_id, round(valor, 2), current_time.strftime('%Y-%m-%d %H:%M:%S'), 
                     estado, random.choice(operador_ids)))
                
                lecturas_generadas += 1
                current_time += timedelta(minutes=intervalo)
        
        print(f"✅ Generadas {lecturas_generadas} lecturas de sensores")
        
        # ⚠️ GENERAR ALARMAS PARA VALORES FUERA DE RANGO
        cursor.execute('''
            SELECT l.id, l.sensor_id, l.valor, l.timestamp, s.nombre, s.valor_min, s.valor_max
            FROM lecturas l
            JOIN sensores s ON l.sensor_id = s.id
            WHERE l.estado != 'NORMAL'
            ORDER BY l.timestamp DESC
            LIMIT 20
        ''')
        
        lecturas_anormales = cursor.fetchall()
        alarmas_creadas = 0
        
        for lectura in lecturas_anormales:
            if lectura['valor'] < lectura['valor_min']:
                tipo_alarma = 'VALOR_BAJO'
                severidad = 'ALTA' if lectura['valor'] < lectura['valor_min'] * 0.8 else 'MEDIA'
                mensaje = f"{lectura['nombre']}: Valor {lectura['valor']} por debajo del mínimo {lectura['valor_min']}"
            else:
                tipo_alarma = 'VALOR_ALTO'
                severidad = 'ALTA' if lectura['valor'] > lectura['valor_max'] * 1.2 else 'MEDIA'
                mensaje = f"{lectura['nombre']}: Valor {lectura['valor']} por encima del máximo {lectura['valor_max']}"
            
            cursor.execute('''
                INSERT INTO alarmas (sensor_id, tipo_alarma, severidad, mensaje, valor_activacion, timestamp_activacion)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (lectura['sensor_id'], tipo_alarma, severidad, mensaje, lectura['valor'], lectura['timestamp']))
            
            alarmas_creadas += 1
        
        print(f"✅ Creadas {alarmas_creadas} alarmas automáticas")
        
        # 💾 CONFIRMAR TODOS LOS CAMBIOS
        conn.commit()
        print("\n🎯 ¡Base de datos poblada con datos industriales realistas!")
        
        return {
            'operadores': len(operadores_data),
            'sensores': len(sensores_data),
            'lecturas': lecturas_generadas,
            'alarmas': alarmas_creadas
        }

# 🚀 EJECUTAR INSERCIÓN DE DATOS
print("🏭 POBLANDO BASE DE DATOS CON DATOS INDUSTRIALES")
print("=" * 55)

try:
    stats = insertar_datos_iniciales()
    
    print(f"\n📊 RESUMEN DE DATOS INSERTADOS:")
    print(f"   👥 Operadores: {stats['operadores']}")
    print(f"   🏭 Sensores: {stats['sensores']}")
    print(f"   📈 Lecturas: {stats['lecturas']}")
    print(f"   ⚠️ Alarmas: {stats['alarmas']}")
    
except Exception as e:
    print(f"❌ Error al insertar datos: {e}")

### 🔵 READ - Consultar y Leer Datos

Ahora que tenemos datos, vamos a practicar diferentes formas de **leer** información usando Python + SQLite.

In [ ]:
# 🔵 READ - CONSULTAS PRÁCTICAS CON PYTHON

def mostrar_dashboard_sensores():
    """Muestra un dashboard de estado actual de sensores"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # 📊 ESTADO ACTUAL DE SENSORES
        print("🏭 DASHBOARD DE SENSORES - ESTADO ACTUAL")
        print("=" * 60)
        
        query = '''
            SELECT 
                s.nombre,
                s.tipo,
                s.ubicacion,
                s.unidad_medida,
                l.valor as ultimo_valor,
                l.estado,
                l.timestamp as ultima_lectura,
                o.nombre || ' ' || o.apellido as operador
            FROM sensores s
            LEFT JOIN lecturas l ON s.id = l.sensor_id
            LEFT JOIN operadores o ON l.operador_id = o.id
            WHERE l.timestamp = (
                SELECT MAX(timestamp) 
                FROM lecturas l2 
                WHERE l2.sensor_id = s.id
            )
            AND s.activo = 1
            ORDER BY s.tipo, s.nombre
        '''
        
        cursor.execute(query)
        sensores_estado = cursor.fetchall()
        
        tipo_actual = ""
        for sensor in sensores_estado:
            # Agrupar por tipo de sensor
            if sensor['tipo'] != tipo_actual:
                tipo_actual = sensor['tipo']
                print(f"\n🔧 {tipo_actual}:")
                print("-" * 40)
            
            # Emoji según estado
            estado_emoji = "🟢" if sensor['estado'] == 'NORMAL' else "🟡" if sensor['estado'] == 'ALTO' else "🔴"
            
            print(f"{estado_emoji} {sensor['nombre']:<25} | {sensor['ultimo_valor']:>6.1f} {sensor['unidad_medida']:<4} | {sensor['ubicacion']}")
            print(f"   📍 Última lectura: {sensor['ultima_lectura']} | 👤 {sensor['operador']}")
        
        return len(sensores_estado)

def obtener_alarmas_activas():
    """Obtiene y muestra alarmas activas del sistema"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        print("\n⚠️ ALARMAS ACTIVAS DEL SISTEMA")
        print("=" * 50)
        
        query = '''
            SELECT 
                a.id,
                s.nombre as sensor,
                a.tipo_alarma,
                a.severidad,
                a.mensaje,
                a.valor_activacion,
                a.timestamp_activacion,
                CASE 
                    WHEN a.timestamp_reconocimiento IS NULL THEN 'NO RECONOCIDA'
                    ELSE 'RECONOCIDA'
                END as estado_reconocimiento
            FROM alarmas a
            JOIN sensores s ON a.sensor_id = s.id
            WHERE a.estado = 'ACTIVA'
            ORDER BY 
                CASE a.severidad 
                    WHEN 'ALTA' THEN 1
                    WHEN 'MEDIA' THEN 2
                    ELSE 3
                END,
                a.timestamp_activacion DESC
        '''
        
        cursor.execute(query)
        alarmas = cursor.fetchall()
        
        if not alarmas:
            print("✅ No hay alarmas activas en el sistema")
            return 0
        
        for alarma in alarmas:
            # Emoji según severidad
            sev_emoji = "🚨" if alarma['severidad'] == 'ALTA' else "⚠️" if alarma['severidad'] == 'MEDIA' else "⚡"
            
            print(f"\n{sev_emoji} ID: {alarma['id']} | {alarma['severidad']} | {alarma['estado_reconocimiento']}")
            print(f"   🏭 Sensor: {alarma['sensor']}")
            print(f"   📝 {alarma['mensaje']}")
            print(f"   🕐 Activada: {alarma['timestamp_activacion']}")
        
        return len(alarmas)

def analizar_tendencias_sensor(sensor_id: int, horas: int = 24):
    """Analiza tendencias de un sensor específico"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # Obtener info del sensor
        cursor.execute("SELECT nombre, tipo, unidad_medida FROM sensores WHERE id = ?", (sensor_id,))
        sensor_info = cursor.fetchone()
        
        if not sensor_info:
            print(f"❌ No se encontró sensor con ID {sensor_id}")
            return
        
        print(f"\n📈 ANÁLISIS DE TENDENCIAS - {sensor_info['nombre']}")
        print("=" * 60)
        
        # Estadísticas de las últimas horas
        query = '''
            SELECT 
                COUNT(*) as total_lecturas,
                ROUND(AVG(valor), 2) as valor_promedio,
                ROUND(MIN(valor), 2) as valor_minimo,
                ROUND(MAX(valor), 2) as valor_maximo,
                SUM(CASE WHEN estado = 'NORMAL' THEN 1 ELSE 0 END) as lecturas_normales,
                SUM(CASE WHEN estado != 'NORMAL' THEN 1 ELSE 0 END) as lecturas_anormales
            FROM lecturas 
            WHERE sensor_id = ? 
            AND timestamp >= datetime('now', '-{} hours')
        '''.format(horas)
        
        cursor.execute(query, (sensor_id,))
        stats = cursor.fetchone()
        
        print(f"📊 Estadísticas últimas {horas} horas:")
        print(f"   📈 Total lecturas: {stats['total_lecturas']}")
        print(f"   📊 Valor promedio: {stats['valor_promedio']} {sensor_info['unidad_medida']}")
        print(f"   📉 Rango: {stats['valor_minimo']} - {stats['valor_maximo']} {sensor_info['unidad_medida']}")
        print(f"   🟢 Lecturas normales: {stats['lecturas_normales']} ({stats['lecturas_normales']/stats['total_lecturas']*100:.1f}%)")
        print(f"   🔴 Lecturas anormales: {stats['lecturas_anormales']} ({stats['lecturas_anormales']/stats['total_lecturas']*100:.1f}%)")
        
        # Últimas 10 lecturas
        query_ultimas = '''
            SELECT valor, estado, timestamp
            FROM lecturas 
            WHERE sensor_id = ?
            ORDER BY timestamp DESC
            LIMIT 10
        '''
        
        cursor.execute(query_ultimas, (sensor_id,))
        ultimas_lecturas = cursor.fetchall()
        
        print(f"\n📋 Últimas 10 lecturas:")
        for lectura in ultimas_lecturas:
            emoji = "🟢" if lectura['estado'] == 'NORMAL' else "🔴"
            print(f"   {emoji} {lectura['valor']:>6.1f} {sensor_info['unidad_medida']} | {lectura['timestamp']}")

# 🚀 EJECUTAR CONSULTAS DE EJEMPLO
print("🔍 EJECUTANDO CONSULTAS DE EJEMPLO")
print("=" * 45)

try:
    # Dashboard de sensores
    total_sensores = mostrar_dashboard_sensores()
    
    # Alarmas activas
    total_alarmas = obtener_alarmas_activas()
    
    # Análisis de tendencias del primer sensor
    print("\n" + "="*60)
    analizar_tendencias_sensor(1, 24)
    
    print(f"\n🎯 RESUMEN:")
    print(f"   🏭 Sensores monitoreados: {total_sensores}")
    print(f"   ⚠️ Alarmas activas: {total_alarmas}")
    
except Exception as e:
    print(f"❌ Error en consultas: {e}")

### 🟡 UPDATE - Actualizar Registros

Las operaciones **UPDATE** son cruciales en sistemas industriales para:
- 📝 Actualizar configuraciones de sensores
- ✅ Resolver alarmas
- 👤 Modificar datos de operadores
- ⚙️ Cambiar parámetros del sistema

In [ ]:
# 🟡 UPDATE - OPERACIONES DE ACTUALIZACIÓN

def reconocer_alarma(alarma_id: int, operador_id: int, observaciones: str = ""):
    """Reconoce una alarma específica por parte de un operador"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # Verificar que la alarma existe y está activa
        cursor.execute('''
            SELECT a.id, s.nombre, a.mensaje 
            FROM alarmas a 
            JOIN sensores s ON a.sensor_id = s.id 
            WHERE a.id = ? AND a.estado = 'ACTIVA'
        ''', (alarma_id,))
        
        alarma = cursor.fetchone()
        if not alarma:
            print(f"❌ No se encontró alarma activa con ID {alarma_id}")
            return False
        
        # Actualizar alarma con reconocimiento
        cursor.execute('''
            UPDATE alarmas 
            SET timestamp_reconocimiento = CURRENT_TIMESTAMP,
                operador_reconocimiento = ?,
                acciones_tomadas = COALESCE(acciones_tomadas, '') || 
                    CASE WHEN acciones_tomadas IS NULL OR acciones_tomadas = '' 
                         THEN ? 
                         ELSE (char(10) || ?) 
                    END
            WHERE id = ?
        ''', (operador_id, f"RECONOCIDA: {observaciones}", f"RECONOCIDA: {observaciones}", alarma_id))
        
        # Obtener nombre del operador
        cursor.execute("SELECT nombre, apellido FROM operadores WHERE id = ?", (operador_id,))
        operador = cursor.fetchone()
        
        conn.commit()
        
        print(f"✅ Alarma #{alarma_id} reconocida exitosamente")
        print(f"   🏭 Sensor: {alarma['nombre']}")
        print(f"   👤 Operador: {operador['nombre']} {operador['apellido']}")
        print(f"   📝 Observaciones: {observaciones}")
        
        return True

def resolver_alarma(alarma_id: int, operador_id: int, solucion: str):
    """Resuelve una alarma marcándola como resuelta"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # Verificar que la alarma existe
        cursor.execute('''
            SELECT a.id, s.nombre, a.mensaje, a.timestamp_reconocimiento
            FROM alarmas a 
            JOIN sensores s ON a.sensor_id = s.id 
            WHERE a.id = ? AND a.estado = 'ACTIVA'
        ''', (alarma_id,))
        
        alarma = cursor.fetchone()
        if not alarma:
            print(f"❌ No se encontró alarma activa con ID {alarma_id}")
            return False
        
        # Actualizar alarma como resuelta
        cursor.execute('''
            UPDATE alarmas 
            SET estado = 'RESUELTA',
                timestamp_resolucion = CURRENT_TIMESTAMP,
                operador_resolucion = ?,
                acciones_tomadas = COALESCE(acciones_tomadas, '') || 
                    CASE WHEN acciones_tomadas IS NULL OR acciones_tomadas = '' 
                         THEN ? 
                         ELSE (char(10) || ?) 
                    END
            WHERE id = ?
        ''', (operador_id, f"RESUELTA: {solucion}", f"RESUELTA: {solucion}", alarma_id))
        
        conn.commit()
        
        print(f"🎯 Alarma #{alarma_id} RESUELTA exitosamente")
        print(f"   📝 Solución aplicada: {solucion}")
        
        return True

def calibrar_sensor(sensor_id: int, operador_id: int):
    """Actualiza la fecha de calibración de un sensor"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # Verificar que el sensor existe
        cursor.execute("SELECT nombre, tipo FROM sensores WHERE id = ?", (sensor_id,))
        sensor = cursor.fetchone()
        
        if not sensor:
            print(f"❌ No se encontró sensor con ID {sensor_id}")
            return False
        
        # Actualizar fecha de calibración
        cursor.execute('''
            UPDATE sensores 
            SET ultima_calibracion = CURRENT_DATE
            WHERE id = ?
        ''', (sensor_id,))
        
        # Registrar la calibración como lectura especial
        cursor.execute('''
            INSERT INTO lecturas (sensor_id, valor, estado, operador_id, observaciones)
            VALUES (?, 0, 'CALIBRACION', ?, 'Sensor calibrado exitosamente')
        ''', (sensor_id, operador_id))
        
        conn.commit()
        
        print(f"🔧 Sensor calibrado exitosamente")
        print(f"   🏭 Sensor: {sensor['nombre']} ({sensor['tipo']})")
        print(f"   📅 Fecha calibración: {datetime.now().strftime('%Y-%m-%d')}")
        
        return True

def actualizar_rango_sensor(sensor_id: int, nuevo_min: float, nuevo_max: float, motivo: str):
    """Actualiza los rangos de operación de un sensor"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # Obtener valores actuales
        cursor.execute('''
            SELECT nombre, valor_min, valor_max, unidad_medida 
            FROM sensores WHERE id = ?
        ''', (sensor_id,))
        
        sensor = cursor.fetchone()
        if not sensor:
            print(f"❌ No se encontró sensor con ID {sensor_id}")
            return False
        
        print(f"🔧 ACTUALIZANDO RANGOS DEL SENSOR: {sensor['nombre']}")
        print(f"   📊 Rango anterior: {sensor['valor_min']} - {sensor['valor_max']} {sensor['unidad_medida']}")
        print(f"   📊 Rango nuevo: {nuevo_min} - {nuevo_max} {sensor['unidad_medida']}")
        print(f"   📝 Motivo: {motivo}")
        
        # Actualizar rangos
        cursor.execute('''
            UPDATE sensores 
            SET valor_min = ?, valor_max = ?
            WHERE id = ?
        ''', (nuevo_min, nuevo_max, sensor_id))
        
        conn.commit()
        
        print("✅ Rangos actualizados exitosamente")
        return True

# 🧪 EJEMPLOS PRÁCTICOS DE UPDATE
print("🟡 DEMOSTRANDO OPERACIONES UPDATE")
print("=" * 45)

try:
    # 1. Reconocer una alarma
    print("1️⃣ RECONOCIENDO ALARMA...")
    reconocer_alarma(1, 1, "Verificando estado del sensor, temperatura dentro de parámetros normales")
    
    print("\n" + "-"*50)
    
    # 2. Resolver una alarma
    print("2️⃣ RESOLVIENDO ALARMA...")
    resolver_alarma(1, 1, "Ajustado setpoint del controlador, problema resuelto")
    
    print("\n" + "-"*50)
    
    # 3. Calibrar sensor
    print("3️⃣ CALIBRANDO SENSOR...")
    calibrar_sensor(1, 2)
    
    print("\n" + "-"*50)
    
    # 4. Actualizar rango de sensor
    print("4️⃣ ACTUALIZANDO RANGO DE SENSOR...")
    actualizar_rango_sensor(2, 1.0, 12.0, "Cambio en especificaciones del proceso")
    
    print("\n🎯 ¡Todas las operaciones UPDATE completadas exitosamente!")
    
except Exception as e:
    print(f"❌ Error en operaciones UPDATE: {e}")

### 🔴 DELETE - Eliminación Segura

En sistemas industriales, **rara vez eliminamos datos** completamente. En su lugar usamos:
- 🔒 **Soft Delete**: Marcar como inactivo
- 📦 **Archivado**: Mover a tablas históricas
- 🗂️ **Retención**: Eliminar datos antiguos según políticas

In [ ]:
# 🔴 DELETE - OPERACIONES DE ELIMINACIÓN SEGURA

def desactivar_sensor(sensor_id: int, motivo: str, operador_id: int):
    """Desactiva un sensor (soft delete) en lugar de eliminarlo"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # Verificar que el sensor existe y está activo
        cursor.execute('''
            SELECT nombre, tipo, ubicacion 
            FROM sensores 
            WHERE id = ? AND activo = 1
        ''', (sensor_id,))
        
        sensor = cursor.fetchone()
        if not sensor:
            print(f"❌ No se encontró sensor activo con ID {sensor_id}")
            return False
        
        # Desactivar sensor (soft delete)
        cursor.execute('''
            UPDATE sensores 
            SET activo = 0
            WHERE id = ?
        ''', (sensor_id,))
        
        # Registrar el motivo de desactivación
        cursor.execute('''
            INSERT INTO lecturas (sensor_id, valor, estado, operador_id, observaciones)
            VALUES (?, 0, 'DESACTIVADO', ?, ?)
        ''', (sensor_id, operador_id, f"Sensor desactivado: {motivo}"))
        
        conn.commit()
        
        print(f"🔒 Sensor desactivado exitosamente")
        print(f"   🏭 Sensor: {sensor['nombre']} ({sensor['tipo']})")
        print(f"   📍 Ubicación: {sensor['ubicacion']}")
        print(f"   📝 Motivo: {motivo}")
        
        return True

def limpiar_datos_antiguos(dias_retention: int = 90):
    """Elimina lecturas antiguas según política de retención"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        print(f"🧹 LIMPIEZA DE DATOS ANTIGUOS (>{dias_retention} días)")
        print("-" * 50)
        
        # Contar registros a eliminar
        cursor.execute('''
            SELECT COUNT(*) as total
            FROM lecturas 
            WHERE timestamp < date('now', '-{} days')
            AND estado != 'CALIBRACION'
        '''.format(dias_retention))
        
        total_a_eliminar = cursor.fetchone()['total']
        
        if total_a_eliminar == 0:
            print("✅ No hay datos antiguos para eliminar")
            return 0
        
        print(f"📊 Se eliminarán {total_a_eliminar} lecturas antiguas")
        
        # Crear tabla de respaldo antes de eliminar
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS lecturas_historicas (
                id INTEGER PRIMARY KEY,
                sensor_id INTEGER,
                valor REAL,
                timestamp DATETIME,
                estado TEXT,
                fecha_archivado DATE DEFAULT CURRENT_DATE
            );
        ''')
        
        # Mover datos a histórico
        cursor.execute('''
            INSERT INTO lecturas_historicas (id, sensor_id, valor, timestamp, estado)
            SELECT id, sensor_id, valor, timestamp, estado
            FROM lecturas 
            WHERE timestamp < date('now', '-{} days')
            AND estado != 'CALIBRACION'
        '''.format(dias_retention))
        
        # Eliminar datos antiguos
        cursor.execute('''
            DELETE FROM lecturas 
            WHERE timestamp < date('now', '-{} days')
            AND estado != 'CALIBRACION'
        '''.format(dias_retention))
        
        registros_eliminados = cursor.rowcount
        
        conn.commit()
        
        print(f"✅ Datos archivados y eliminados exitosamente")
        print(f"   📦 Movidos a histórico: {total_a_eliminar}")
        print(f"   🗑️ Eliminados de tabla principal: {registros_eliminados}")
        
        return registros_eliminados

def eliminar_alarmas_resueltas_antiguas(dias: int = 30):
    """Elimina alarmas resueltas que tienen más de X días"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # Contar alarmas a eliminar
        cursor.execute('''
            SELECT COUNT(*) as total
            FROM alarmas 
            WHERE estado = 'RESUELTA' 
            AND timestamp_resolucion < date('now', '-{} days')
        '''.format(dias))
        
        total_alarmas = cursor.fetchone()['total']
        
        if total_alarmas == 0:
            print(f"✅ No hay alarmas resueltas de más de {dias} días")
            return 0
        
        print(f"🧹 ELIMINANDO {total_alarmas} ALARMAS RESUELTAS ANTIGUAS")
        print("-" * 55)
        
        # Eliminar alarmas resueltas antiguas
        cursor.execute('''
            DELETE FROM alarmas 
            WHERE estado = 'RESUELTA' 
            AND timestamp_resolucion < date('now', '-{} days')
        '''.format(dias))
        
        registros_eliminados = cursor.rowcount
        
        conn.commit()
        
        print(f"✅ Eliminadas {registros_eliminados} alarmas resueltas antiguas")
        
        return registros_eliminados

def desactivar_operador(operador_id: int, motivo: str):
    """Desactiva un operador en lugar de eliminarlo"""
    
    with obtener_conexion_db() as conn:
        cursor = conn.cursor()
        
        # Verificar que el operador existe
        cursor.execute('''
            SELECT nombre, apellido, usuario 
            FROM operadores 
            WHERE id = ? AND activo = 1
        ''', (operador_id,))
        
        operador = cursor.fetchone()
        if not operador:
            print(f"❌ No se encontró operador activo con ID {operador_id}")
            return False
        
        # Desactivar operador
        cursor.execute('''
            UPDATE operadores 
            SET activo = 0
            WHERE id = ?
        ''', (operador_id,))
        
        conn.commit()
        
        print(f"🔒 Operador desactivado exitosamente")
        print(f"   👤 Operador: {operador['nombre']} {operador['apellido']} (@{operador['usuario']})")
        print(f"   📝 Motivo: {motivo}")
        
        return True

# 🧪 EJEMPLOS PRÁCTICOS DE DELETE
print("🔴 DEMOSTRANDO OPERACIONES DELETE SEGURAS")
print("=" * 50)

try:
    # 1. Limpiar datos antiguos (simulado con 1000 días para no eliminar nada en demo)
    print("1️⃣ VERIFICANDO DATOS ANTIGUOS...")
    eliminados = limpiar_datos_antiguos(1000)  # Usar 1000 días para no eliminar en demo
    
    print("\n" + "-"*50)
    
    # 2. Eliminar alarmas resueltas antiguas (simulado)
    print("2️⃣ VERIFICANDO ALARMAS RESUELTAS ANTIGUAS...")
    alarmas_eliminadas = eliminar_alarmas_resueltas_antiguas(1000)  # Usar 1000 días para demo
    
    print("\n" + "-"*50)
    
    # 3. Demostrar desactivación de sensor (comentado para no afectar demo)
    print("3️⃣ EJEMPLO DE DESACTIVACIÓN DE SENSOR...")
    print("   (Comentado para mantener integridad de datos en demo)")
    # desactivar_sensor(8, "Sensor defectuoso, pendiente reemplazo", 1)
    
    print("\n🎯 ¡Operaciones DELETE seguras completadas!")
    print("💡 Nota: En sistemas industriales, preferimos desactivar en lugar de eliminar")
    
except Exception as e:
    print(f"❌ Error en operaciones DELETE: {e}")

## 🏗️ 4. PATRONES DE DISEÑO DAO

### 🎯 ¿Qué es el patrón DAO?

**DAO (Data Access Object)** es un patrón de diseño que encapsula todas las operaciones de base de datos en clases especializadas.

**🔥 VENTAJAS:**
- 📦 **Encapsulación**: Toda la lógica de BD en un lugar
- 🔧 **Mantenibilidad**: Cambios centralizados
- 🧪 **Testeable**: Fácil de probar unitariamente
- 🔄 **Reutilizable**: Usar en múltiples partes del código
- 🛡️ **Seguro**: Manejo consistente de errores

In [ ]:
# 🏗️ IMPLEMENTACIÓN DEL PATRÓN DAO

from typing import List, Dict, Any, Optional
from dataclasses import dataclass

@dataclass
class Sensor:
    """Clase de datos para representar un sensor"""
    id: Optional[int] = None
    nombre: str = ""
    tipo: str = ""
    ubicacion: str = ""
    unidad_medida: str = ""
    valor_min: float = 0.0
    valor_max: float = 100.0
    activo: bool = True
    fecha_instalacion: str = ""
    ultima_calibracion: Optional[str] = None
    intervalo_lectura: int = 60
    descripcion: str = ""

class SensorDAO:
    """Data Access Object para operaciones de sensores"""
    
    def __init__(self, db_path: str = 'sistema_industrial.db'):
        self.db_path = db_path
    
    def crear_sensor(self, sensor: Sensor) -> int:
        """Crea un nuevo sensor en la base de datos"""
        
        with obtener_conexion_db(self.db_path) as conn:
            cursor = conn.cursor()
            
            cursor.execute('''
                INSERT INTO sensores (nombre, tipo, ubicacion, unidad_medida, 
                                    valor_min, valor_max, fecha_instalacion, 
                                    intervalo_lectura, descripcion)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                sensor.nombre, sensor.tipo, sensor.ubicacion, sensor.unidad_medida,
                sensor.valor_min, sensor.valor_max, sensor.fecha_instalacion,
                sensor.intervalo_lectura, sensor.descripcion
            ))
            
            conn.commit()
            sensor_id = cursor.lastrowid
            
            print(f"✅ Sensor creado con ID: {sensor_id}")
            return sensor_id
    
    def obtener_sensor(self, sensor_id: int) -> Optional[Sensor]:
        """Obtiene un sensor por su ID"""
        
        with obtener_conexion_db(self.db_path) as conn:
            cursor = conn.cursor()
            
            cursor.execute("SELECT * FROM sensores WHERE id = ?", (sensor_id,))
            row = cursor.fetchone()
            
            if not row:
                return None
            
            return Sensor(
                id=row['id'],
                nombre=row['nombre'],
                tipo=row['tipo'],
                ubicacion=row['ubicacion'],
                unidad_medida=row['unidad_medida'],
                valor_min=row['valor_min'],
                valor_max=row['valor_max'],
                activo=bool(row['activo']),
                fecha_instalacion=row['fecha_instalacion'],
                ultima_calibracion=row['ultima_calibracion'],
                intervalo_lectura=row['intervalo_lectura'],
                descripcion=row['descripcion']
            )
    
    def obtener_todos_sensores(self, solo_activos: bool = True) -> List[Sensor]:
        """Obtiene todos los sensores"""
        
        with obtener_conexion_db(self.db_path) as conn:
            cursor = conn.cursor()
            
            query = "SELECT * FROM sensores"
            if solo_activos:
                query += " WHERE activo = 1"
            query += " ORDER BY tipo, nombre"
            
            cursor.execute(query)
            rows = cursor.fetchall()
            
            sensores = []
            for row in rows:
                sensor = Sensor(
                    id=row['id'],
                    nombre=row['nombre'],
                    tipo=row['tipo'],
                    ubicacion=row['ubicacion'],
                    unidad_medida=row['unidad_medida'],
                    valor_min=row['valor_min'],
                    valor_max=row['valor_max'],
                    activo=bool(row['activo']),
                    fecha_instalacion=row['fecha_instalacion'],
                    ultima_calibracion=row['ultima_calibracion'],
                    intervalo_lectura=row['intervalo_lectura'],
                    descripcion=row['descripcion']
                )
                sensores.append(sensor)
            
            return sensores
    
    def actualizar_sensor(self, sensor: Sensor) -> bool:
        """Actualiza un sensor existente"""
        
        if not sensor.id:
            print("❌ El sensor debe tener un ID para actualizar")
            return False
        
        with obtener_conexion_db(self.db_path) as conn:
            cursor = conn.cursor()
            
            cursor.execute('''
                UPDATE sensores 
                SET nombre = ?, tipo = ?, ubicacion = ?, unidad_medida = ?,
                    valor_min = ?, valor_max = ?, intervalo_lectura = ?, 
                    descripcion = ?
                WHERE id = ?
            ''', (
                sensor.nombre, sensor.tipo, sensor.ubicacion, sensor.unidad_medida,
                sensor.valor_min, sensor.valor_max, sensor.intervalo_lectura,
                sensor.descripcion, sensor.id
            ))
            
            if cursor.rowcount == 0:
                print(f"❌ No se encontró sensor con ID {sensor.id}")
                return False
            
            conn.commit()
            print(f"✅ Sensor {sensor.id} actualizado exitosamente")
            return True
    
    def buscar_sensores(self, filtros: Dict[str, Any]) -> List[Sensor]:
        """Busca sensores con filtros específicos"""
        
        with obtener_conexion_db(self.db_path) as conn:
            cursor = conn.cursor()
            
            where_clauses = []
            valores = []
            
            for campo, valor in filtros.items():
                if campo == 'tipo' and valor:
                    where_clauses.append("tipo = ?")
                    valores.append(valor)
                elif campo == 'ubicacion' and valor:
                    where_clauses.append("ubicacion LIKE ?")
                    valores.append(f"%{valor}%")
                elif campo == 'activo':
                    where_clauses.append("activo = ?")
                    valores.append(1 if valor else 0)
            
            query = "SELECT * FROM sensores"
            if where_clauses:
                query += " WHERE " + " AND ".join(where_clauses)
            query += " ORDER BY nombre"
            
            cursor.execute(query, valores)
            rows = cursor.fetchall()
            
            sensores = []
            for row in rows:
                sensor = Sensor(
                    id=row['id'],
                    nombre=row['nombre'],
                    tipo=row['tipo'],
                    ubicacion=row['ubicacion'],
                    unidad_medida=row['unidad_medida'],
                    valor_min=row['valor_min'],
                    valor_max=row['valor_max'],
                    activo=bool(row['activo']),
                    fecha_instalacion=row['fecha_instalacion'],
                    ultima_calibracion=row['ultima_calibracion'],
                    intervalo_lectura=row['intervalo_lectura'],
                    descripcion=row['descripcion']
                )
                sensores.append(sensor)
            
            return sensores
    
    def obtener_estadisticas_por_tipo(self) -> Dict[str, int]:
        """Obtiene estadísticas de sensores por tipo"""
        
        with obtener_conexion_db(self.db_path) as conn:
            cursor = conn.cursor()
            
            cursor.execute('''
                SELECT tipo, COUNT(*) as cantidad
                FROM sensores 
                WHERE activo = 1
                GROUP BY tipo
                ORDER BY cantidad DESC
            ''')
            
            stats = {}
            for row in cursor.fetchall():
                stats[row['tipo']] = row['cantidad']
            
            return stats

# 🧪 DEMO DEL PATRÓN DAO
print("🏗️ DEMOSTRANDO PATRÓN DAO")
print("=" * 35)

try:
    # Crear instancia del DAO
    sensor_dao = SensorDAO()
    
    # 1. Obtener todos los sensores
    print("1️⃣ OBTENIENDO TODOS LOS SENSORES...")
    sensores = sensor_dao.obtener_todos_sensores()
    print(f"   📊 Total sensores activos: {len(sensores)}")
    
    # 2. Obtener un sensor específico
    print("\n2️⃣ OBTENIENDO SENSOR ESPECÍFICO...")
    sensor1 = sensor_dao.obtener_sensor(1)
    if sensor1:
        print(f"   🏭 Sensor: {sensor1.nombre} ({sensor1.tipo})")
        print(f"   📍 Ubicación: {sensor1.ubicacion}")
        print(f"   📊 Rango: {sensor1.valor_min} - {sensor1.valor_max} {sensor1.unidad_medida}")
    
    # 3. Buscar sensores por tipo
    print("\n3️⃣ BUSCANDO SENSORES DE TEMPERATURA...")
    sensores_temp = sensor_dao.buscar_sensores({'tipo': 'TEMPERATURA'})
    for sensor in sensores_temp:
        print(f"   🌡️ {sensor.nombre} - {sensor.ubicacion}")
    
    # 4. Estadísticas por tipo
    print("\n4️⃣ ESTADÍSTICAS POR TIPO...")
    stats = sensor_dao.obtener_estadisticas_por_tipo()
    for tipo, cantidad in stats.items():
        print(f"   📊 {tipo}: {cantidad} sensores")
    
    # 5. Crear un nuevo sensor usando DAO
    print("\n5️⃣ CREANDO NUEVO SENSOR CON DAO...")
    nuevo_sensor = Sensor(
        nombre="Temperatura Horno 2",
        tipo="TEMPERATURA",
        ubicacion="Horno Industrial - Sector E",
        unidad_medida="°C",
        valor_min=100.0,
        valor_max=500.0,
        fecha_instalacion=datetime.now().strftime('%Y-%m-%d'),
        intervalo_lectura=30,
        descripcion="Sensor de temperatura crítica para horno industrial"
    )
    
    nuevo_id = sensor_dao.crear_sensor(nuevo_sensor)
    print(f"   ✅ Nuevo sensor creado con ID: {nuevo_id}")
    
    print("\n🎯 ¡Patrón DAO implementado exitosamente!")
    print("💡 Beneficios: Código organizado, reutilizable y mantenible")
    
except Exception as e:
    print(f"❌ Error en demostración DAO: {e}")

## 6. 🐼 Integración con Pandas: Análisis Híbrido de Datos

La integración de SQLite con Pandas nos permite combinar el poder de las consultas SQL con las capacidades de análisis de datos de Pandas. Esto es especialmente útil en sistemas industriales donde necesitamos generar reportes, análisis estadísticos y visualizaciones.

### 6.1 Conexión SQLite + Pandas

Pandas puede leer directamente desde SQLite y también escribir DataFrames a la base de datos.

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import numpy as np

# Configuración para visualizaciones
plt.style.use('default')
sns.set_palette("husl")

print("📊 Iniciando análisis híbrido SQLite + Pandas...")

# Redefinir función de conexión para esta sección
def get_db_connection():
    """Context manager para conexiones a SQLite"""
    import sqlite3
    import contextlib
    
    @contextlib.contextmanager  
    def db_connection():
        conn = None
        try:
            conn = sqlite3.connect('sistema_industrial.db')  # Usar la nueva base
            conn.row_factory = sqlite3.Row
            yield conn
            conn.commit()
        except Exception as e:
            if conn:
                conn.rollback()
            raise e
        finally:
            if conn:
                conn.close()
    
    return db_connection()

# 1. LECTURA DIRECTA DESDE SQLITE CON PANDAS
def analizar_datos_sensores():
    """Análisis completo de datos de sensores usando Pandas"""
    
    with get_db_connection() as conn:
        # Leer datos de sensores con JOIN
        query_sensores = """
        SELECT 
            s.nombre as sensor,
            s.tipo,
            s.ubicacion,
            l.valor,
            l.timestamp,
            l.calidad
        FROM sensores s
        JOIN lecturas l ON s.id = l.sensor_id
        ORDER BY l.timestamp DESC
        """
        
        df_lecturas = pd.read_sql_query(query_sensores, conn)
        
        print(f"📈 DataFrame cargado: {len(df_lecturas)} registros")
        print("\n🔍 Primeras 5 filas:")
        print(df_lecturas.head())
        
        return df_lecturas

# Ejecutar análisis
df_sensores = analizar_datos_sensores()

📊 Iniciando análisis híbrido SQLite + Pandas...
📈 DataFrame cargado: 100 registros

🔍 Primeras 5 filas:
           sensor         tipo ubicacion  valor                   timestamp  \
0   TEMP_MOTOR_M2  temperatura  Motor M2  43.27  2025-07-05T08:01:22.742684   
1   PRES_BOMBA_A1      presion  Bomba A1  16.78  2025-07-05T08:01:22.742556   
2   TEMP_MOTOR_M2  temperatura  Motor M2  68.34  2025-07-05T07:01:22.742659   
3  NIVEL_TANQUE_B        nivel  Tanque B  75.97  2025-07-05T07:01:22.742639   
4   PRES_BOMBA_A1      presion  Bomba A1  41.53  2025-07-05T07:01:22.742554   

   calidad  
0     MALA  
1     MALA  
2  REGULAR  
3    BUENA  
4    BUENA  


In [15]:
# 2. ANÁLISIS ESTADÍSTICO AVANZADO
def analisis_estadistico_completo(df):
    """Análisis estadístico completo de los datos de sensores"""
    
    print("📊 ANÁLISIS ESTADÍSTICO COMPLETO")
    print("=" * 50)
    
    # Convertir timestamp a datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Estadísticas descriptivas por sensor
    print("\n📈 Estadísticas por Sensor:")
    estadisticas = df.groupby(['sensor', 'tipo'])['valor'].agg([
        'count', 'mean', 'std', 'min', 'max',
        lambda x: x.quantile(0.25),  # Q1
        lambda x: x.quantile(0.75),  # Q3
    ]).round(2)
    
    estadisticas.columns = ['Lecturas', 'Promedio', 'Desv_Std', 'Mínimo', 'Máximo', 'Q1', 'Q3']
    print(estadisticas)
    
    # Detección de valores atípicos (outliers)
    print("\n🚨 Detección de Valores Atípicos:")
    for sensor in df['sensor'].unique():
        datos_sensor = df[df['sensor'] == sensor]['valor']
        Q1 = datos_sensor.quantile(0.25)
        Q3 = datos_sensor.quantile(0.75)
        IQR = Q3 - Q1
        
        # Límites para outliers
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        
        outliers = datos_sensor[(datos_sensor < limite_inferior) | (datos_sensor > limite_superior)]
        
        if len(outliers) > 0:
            print(f"  🔴 {sensor}: {len(outliers)} valores atípicos detectados")
            print(f"     Rango normal: [{limite_inferior:.2f}, {limite_superior:.2f}]")
            print(f"     Valores atípicos: {outliers.tolist()}")
        else:
            print(f"  ✅ {sensor}: Sin valores atípicos")
    
    # Análisis de tendencias temporales
    print("\n📈 Análisis de Tendencias (últimas 24 horas):")
    df_reciente = df[df['timestamp'] >= df['timestamp'].max() - timedelta(hours=24)]
    
    for sensor in df_reciente['sensor'].unique():
        datos_sensor = df_reciente[df_reciente['sensor'] == sensor]
        if len(datos_sensor) > 1:
            # Calcular tendencia (correlación con tiempo)
            datos_sensor = datos_sensor.sort_values('timestamp')
            datos_sensor['tiempo_numerico'] = (datos_sensor['timestamp'] - datos_sensor['timestamp'].min()).dt.total_seconds()
            
            correlacion = datos_sensor['valor'].corr(datos_sensor['tiempo_numerico'])
            
            if correlacion > 0.5:
                tendencia = "📈 AUMENTANDO"
            elif correlacion < -0.5:
                tendencia = "📉 DISMINUYENDO"
            else:
                tendencia = "📊 ESTABLE"
                
            print(f"  {sensor}: {tendencia} (r={correlacion:.3f})")
    
    return estadisticas

# Ejecutar análisis estadístico
stats = analisis_estadistico_completo(df_sensores)

📊 ANÁLISIS ESTADÍSTICO COMPLETO

📈 Estadísticas por Sensor:
                             Lecturas  Promedio  Desv_Std  Mínimo  Máximo  \
sensor          tipo                                                        
FLUJO_LINEA_3   flujo              20    115.60     29.75   69.10  179.49   
NIVEL_TANQUE_B  nivel              20     59.50     22.05   24.89   92.20   
PRES_BOMBA_A1   presion            20     30.06      9.60   15.59   44.12   
TEMP_MOTOR_M2   temperatura        20     51.04      9.03   40.46   69.58   
TEMP_REACTOR_01 temperatura        20    212.94     33.76  162.28  275.06   

                                 Q1      Q3  
sensor          tipo                         
FLUJO_LINEA_3   flujo         90.60  132.22  
NIVEL_TANQUE_B  nivel         40.69   76.92  
PRES_BOMBA_A1   presion       24.11   38.97  
TEMP_MOTOR_M2   temperatura   45.06   56.72  
TEMP_REACTOR_01 temperatura  189.24  232.54  

🚨 Detección de Valores Atípicos:
  ✅ TEMP_MOTOR_M2: Sin valores atípicos
  ✅ 

In [16]:
# 3. GENERACIÓN AUTOMÁTICA DE REPORTES
def generar_reporte_operacional():
    """Genera un reporte operacional completo del sistema industrial"""
    
    with get_db_connection() as conn:
        # Datos para el reporte
        reporte_data = {}
        
        # 1. Estado general del sistema
        query_resumen = """
        SELECT 
            COUNT(DISTINCT s.id) as total_sensores,
            COUNT(DISTINCT s.id) FILTER (WHERE s.activo = 1) as sensores_activos,
            COUNT(l.id) as total_lecturas,
            COUNT(DISTINCT a.id) as total_alarmas,
            COUNT(DISTINCT a.id) FILTER (WHERE a.reconocida = 0) as alarmas_pendientes
        FROM sensores s
        LEFT JOIN lecturas l ON s.id = l.sensor_id AND l.timestamp >= datetime('now', '-24 hours')
        LEFT JOIN alarmas a ON s.id = a.sensor_id AND a.timestamp >= datetime('now', '-24 hours')
        """
        
        resumen = pd.read_sql_query(query_resumen, conn).iloc[0]
        reporte_data['resumen'] = resumen
        
        # 2. Top sensores con más lecturas
        query_top_sensores = """
        SELECT 
            s.nombre,
            s.tipo,
            s.ubicacion,
            COUNT(l.id) as num_lecturas,
            AVG(l.valor) as valor_promedio,
            MAX(l.timestamp) as ultima_lectura
        FROM sensores s
        JOIN lecturas l ON s.id = l.sensor_id
        WHERE l.timestamp >= datetime('now', '-24 hours')
        GROUP BY s.id, s.nombre, s.tipo, s.ubicacion
        ORDER BY num_lecturas DESC
        LIMIT 5
        """
        
        top_sensores = pd.read_sql_query(query_top_sensores, conn)
        reporte_data['top_sensores'] = top_sensores
        
        # 3. Alarmas críticas
        query_alarmas = """
        SELECT 
            s.nombre as sensor,
            a.tipo_alarma,
            a.mensaje,
            a.timestamp,
            a.reconocida
        FROM alarmas a
        JOIN sensores s ON a.sensor_id = s.id
        WHERE a.timestamp >= datetime('now', '-24 hours')
        ORDER BY a.timestamp DESC
        LIMIT 10
        """
        
        alarmas_recientes = pd.read_sql_query(query_alarmas, conn)
        reporte_data['alarmas'] = alarmas_recientes
        
        return reporte_data

def exportar_reporte_excel(reporte_data, nombre_archivo="reporte_operacional.xlsx"):
    """Exporta el reporte a un archivo Excel con múltiples hojas"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"reporte_operacional_{timestamp}.xlsx"
    
    with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
        # Hoja 1: Resumen ejecutivo
        df_resumen = pd.DataFrame([reporte_data['resumen']])
        df_resumen.to_excel(writer, sheet_name='Resumen_Ejecutivo', index=False)
        
        # Hoja 2: Top sensores
        reporte_data['top_sensores'].to_excel(writer, sheet_name='Top_Sensores', index=False)
        
        # Hoja 3: Alarmas recientes
        reporte_data['alarmas'].to_excel(writer, sheet_name='Alarmas_Recientes', index=False)
        
        # Hoja 4: Análisis detallado
        if 'df_sensores' in globals():
            df_sensores.to_excel(writer, sheet_name='Datos_Detallados', index=False)
    
    print(f"📄 Reporte exportado: {nombre_archivo}")
    return nombre_archivo

# Generar y exportar reporte
print("📋 Generando reporte operacional...")
reporte = generar_reporte_operacional()

print("\n📊 RESUMEN EJECUTIVO:")
print(f"  • Total sensores: {reporte['resumen']['total_sensores']}")
print(f"  • Sensores activos: {reporte['resumen']['sensores_activos']}")
print(f"  • Lecturas (24h): {reporte['resumen']['total_lecturas']}")
print(f"  • Alarmas (24h): {reporte['resumen']['total_alarmas']}")
print(f"  • Alarmas pendientes: {reporte['resumen']['alarmas_pendientes']}")

print("\n🏆 TOP 5 SENSORES MÁS ACTIVOS:")
print(reporte['top_sensores'][['nombre', 'tipo', 'num_lecturas', 'valor_promedio']].to_string(index=False))

print("\n🚨 ALARMAS RECIENTES:")
if len(reporte['alarmas']) > 0:
    print(reporte['alarmas'][['sensor', 'tipo_alarma', 'mensaje', 'reconocida']].head().to_string(index=False))
else:
    print("  ✅ No hay alarmas recientes")

# Exportar a Excel
archivo_reporte = exportar_reporte_excel(reporte)

📋 Generando reporte operacional...

📊 RESUMEN EJECUTIVO:
  • Total sensores: 5
  • Sensores activos: 5
  • Lecturas (24h): 46
  • Alarmas (24h): 3
  • Alarmas pendientes: 3

🏆 TOP 5 SENSORES MÁS ACTIVOS:
         nombre        tipo  num_lecturas  valor_promedio
  PRES_BOMBA_A1     presion            13       32.001538
 NIVEL_TANQUE_B       nivel             9       53.880000
TEMP_REACTOR_01 temperatura             8      205.082500
  FLUJO_LINEA_3       flujo             8      109.206250
  TEMP_MOTOR_M2 temperatura             8       51.505000

🚨 ALARMAS RECIENTES:
         sensor      tipo_alarma                                 mensaje  reconocida
TEMP_REACTOR_01 TEMPERATURA_ALTA Temperatura reactor por encima de 280°C           0
  PRES_BOMBA_A1     PRESION_BAJA      Presión bomba por debajo de 15 bar           0
  FLUJO_LINEA_3  FLUJO_IRREGULAR        Fluctuaciones anormales en flujo           0
📄 Reporte exportado: reporte_operacional_20250705_090214.xlsx
📄 Reporte exportado: rep

## 7. 💪 Ejercicios Prácticos Progresivos

### Nivel Básico 🟢

**Ejercicio 1: Conexión y Consultas Básicas**
- Crear una conexión a SQLite
- Realizar consultas SELECT simples
- Contar registros y filtrar datos

**Ejercicio 2: Inserción de Datos**
- Insertar nuevos sensores y lecturas
- Manejar errores de inserción
- Validar datos antes de insertar

In [17]:
# 🟢 EJERCICIOS NIVEL BÁSICO

print("💪 EJERCICIOS PRÁCTICOS - NIVEL BÁSICO")
print("=" * 50)

# EJERCICIO 1: Conexión y Consultas Básicas
def ejercicio_1_consultas_basicas():
    """
    EJERCICIO 1: Realizar consultas básicas a la base de datos
    
    TAREAS:
    1. Conectar a la base de datos
    2. Contar total de sensores
    3. Listar sensores activos
    4. Mostrar últimas 5 lecturas
    """
    print("\n🎯 EJERCICIO 1: Consultas Básicas")
    
    # TU CÓDIGO AQUÍ:
    with get_db_connection() as conn:
        cursor = conn.cursor()
        
        # 1. Contar total de sensores
        cursor.execute("SELECT COUNT(*) FROM sensores")
        total_sensores = cursor.fetchone()[0]
        print(f"  📊 Total de sensores: {total_sensores}")
        
        # 2. Listar sensores activos
        cursor.execute("SELECT nombre, tipo, ubicacion FROM sensores WHERE activo = 1")
        sensores_activos = cursor.fetchall()
        print(f"  ✅ Sensores activos: {len(sensores_activos)}")
        for sensor in sensores_activos:
            print(f"    - {sensor[0]} ({sensor[1]}) en {sensor[2]}")
        
        # 3. Últimas 5 lecturas
        cursor.execute("""
            SELECT s.nombre, l.valor, l.timestamp 
            FROM lecturas l 
            JOIN sensores s ON l.sensor_id = s.id 
            ORDER BY l.timestamp DESC 
            LIMIT 5
        """)
        ultimas_lecturas = cursor.fetchall()
        print(f"  📈 Últimas 5 lecturas:")
        for lectura in ultimas_lecturas:
            print(f"    - {lectura[0]}: {lectura[1]} ({lectura[2]})")

# EJERCICIO 2: Inserción de Datos
def ejercicio_2_insercion_datos():
    """
    EJERCICIO 2: Insertar nuevos datos en la base
    
    TAREAS:
    1. Insertar un nuevo sensor
    2. Agregar lecturas para el sensor
    3. Manejar errores de inserción
    """
    print("\n🎯 EJERCICIO 2: Inserción de Datos")
    
    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()
            
            # 1. Insertar nuevo sensor
            nuevo_sensor = (
                'SENSOR_EJERCICIO_01',
                'flujo',
                'Línea de Producción C',
                'Sensor de práctica para ejercicios',
                0.0,  # rango_min
                100.0,  # rango_max
                1  # activo
            )
            
            cursor.execute("""
                INSERT OR IGNORE INTO sensores 
                (nombre, tipo, ubicacion, descripcion, rango_min, rango_max, activo)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, nuevo_sensor)
            
            # Obtener ID del sensor
            cursor.execute("SELECT id FROM sensores WHERE nombre = ?", ('SENSOR_EJERCICIO_01',))
            sensor_id = cursor.fetchone()[0]
            
            print(f"  ✅ Sensor creado con ID: {sensor_id}")
            
            # 2. Agregar lecturas
            import random
            lecturas = []
            for i in range(5):
                lectura = (
                    sensor_id,
                    round(random.uniform(10, 90), 2),
                    datetime.now().isoformat(),
                    'BUENA'
                )
                lecturas.append(lectura)
            
            cursor.executemany("""
                INSERT INTO lecturas (sensor_id, valor, timestamp, calidad)
                VALUES (?, ?, ?, ?)
            """, lecturas)
            
            print(f"  📊 {len(lecturas)} lecturas agregadas")
            
            conn.commit()
            print("  💾 Datos guardados exitosamente")
            
    except Exception as e:
        print(f"  ❌ Error: {e}")

# Ejecutar ejercicios básicos
ejercicio_1_consultas_basicas()
ejercicio_2_insercion_datos()

print("\n✅ Ejercicios básicos completados. ¡Continúa con el nivel intermedio!")

💪 EJERCICIOS PRÁCTICOS - NIVEL BÁSICO

🎯 EJERCICIO 1: Consultas Básicas
  📊 Total de sensores: 5
  ✅ Sensores activos: 5
    - TEMP_REACTOR_01 (temperatura) en Reactor Principal
    - PRES_BOMBA_A1 (presion) en Bomba A1
    - FLUJO_LINEA_3 (flujo) en Línea Producción 3
    - NIVEL_TANQUE_B (nivel) en Tanque B
    - TEMP_MOTOR_M2 (temperatura) en Motor M2
  📈 Últimas 5 lecturas:
    - TEMP_MOTOR_M2: 43.27 (2025-07-05T08:01:22.742684)
    - PRES_BOMBA_A1: 16.78 (2025-07-05T08:01:22.742556)
    - TEMP_MOTOR_M2: 68.34 (2025-07-05T07:01:22.742659)
    - NIVEL_TANQUE_B: 75.97 (2025-07-05T07:01:22.742639)
    - PRES_BOMBA_A1: 41.53 (2025-07-05T07:01:22.742554)

🎯 EJERCICIO 2: Inserción de Datos
  ❌ Error: table sensores has no column named descripcion

✅ Ejercicios básicos completados. ¡Continúa con el nivel intermedio!


### Nivel Intermedio 🟡

**Ejercicio 3: Análisis con Pandas**
- Cargar datos con pandas.read_sql_query()
- Realizar análisis estadístico básico
- Detectar valores atípicos

**Ejercicio 4: Gestión de Alarmas**
- Implementar sistema de detección de alarmas
- Crear y gestionar alarmas automáticamente
- Generar reportes de alarmas

In [ ]:
# 🟡 EJERCICIOS NIVEL INTERMEDIO

print("💪 EJERCICIOS PRÁCTICOS - NIVEL INTERMEDIO")
print("=" * 50)

# EJERCICIO 3: Análisis con Pandas
def ejercicio_3_analisis_pandas():
    """
    EJERCICIO 3: Análisis avanzado usando Pandas
    
    TAREAS:
    1. Cargar datos con pandas
    2. Análisis estadístico por tipo de sensor
    3. Detectar valores atípicos
    4. Crear visualización básica
    """
    print("\n🎯 EJERCICIO 3: Análisis con Pandas")
    
    with get_db_connection() as conn:
        # 1. Cargar datos
        query = """
        SELECT 
            s.nombre as sensor,
            s.tipo,
            s.ubicacion,
            l.valor,
            l.timestamp,
            l.calidad
        FROM sensores s
        JOIN lecturas l ON s.id = l.sensor_id
        WHERE l.timestamp >= datetime('now', '-3 days')
        """
        
        df = pd.read_sql_query(query, conn)
        print(f"  📊 Datos cargados: {len(df)} registros")
        
        # 2. Análisis estadístico por tipo
        print("\n  📈 Estadísticas por tipo de sensor:")
        stats_por_tipo = df.groupby('tipo')['valor'].agg([
            'count', 'mean', 'std', 'min', 'max'
        ]).round(2)
        print(stats_por_tipo)
        
        # 3. Detección de valores atípicos
        print("\n  🚨 Detección de valores atípicos:")
        for tipo in df['tipo'].unique():
            datos_tipo = df[df['tipo'] == tipo]['valor']
            Q1 = datos_tipo.quantile(0.25)
            Q3 = datos_tipo.quantile(0.75)
            IQR = Q3 - Q1
            
            outliers = datos_tipo[
                (datos_tipo < Q1 - 1.5 * IQR) | 
                (datos_tipo > Q3 + 1.5 * IQR)
            ]
            
            print(f"    {tipo}: {len(outliers)} valores atípicos")
        
        return df

# EJERCICIO 4: Gestión de Alarmas
def ejercicio_4_gestion_alarmas():
    """
    EJERCICIO 4: Sistema completo de gestión de alarmas
    
    TAREAS:
    1. Detectar condiciones de alarma
    2. Crear alarmas automáticamente
    3. Gestionar estado de alarmas
    4. Generar reporte de alarmas
    """
    print("\n🎯 EJERCICIO 4: Gestión de Alarmas")
    
    def detectar_alarmas_automaticas():
        """Detecta y crea alarmas basadas en reglas"""
        with get_db_connection() as conn:
            cursor = conn.cursor()
            
            # Buscar lecturas fuera de rango
            cursor.execute("""
                SELECT 
                    s.id, s.nombre, s.rango_min, s.rango_max, 
                    l.valor, l.timestamp
                FROM sensores s
                JOIN lecturas l ON s.id = l.sensor_id
                WHERE (l.valor < s.rango_min OR l.valor > s.rango_max)
                AND l.timestamp >= datetime('now', '-1 hour')
                AND s.activo = 1
            """)
            
            lecturas_problematicas = cursor.fetchall()
            alarmas_creadas = 0
            
            for lectura in lecturas_problematicas:
                sensor_id, nombre, rango_min, rango_max, valor, timestamp = lectura
                
                # Determinar tipo de alarma
                if valor < rango_min:
                    tipo_alarma = "VALOR_BAJO"
                    mensaje = f"Valor {valor} por debajo del rango mínimo {rango_min}"
                else:
                    tipo_alarma = "VALOR_ALTO"
                    mensaje = f"Valor {valor} por encima del rango máximo {rango_max}"
                
                # Verificar si ya existe alarma similar reciente
                cursor.execute("""
                    SELECT COUNT(*) FROM alarmas 
                    WHERE sensor_id = ? 
                    AND tipo_alarma = ? 
                    AND timestamp >= datetime('now', '-30 minutes')
                """, (sensor_id, tipo_alarma))
                
                if cursor.fetchone()[0] == 0:  # No hay alarma similar reciente
                    # Crear nueva alarma
                    cursor.execute("""
                        INSERT INTO alarmas 
                        (sensor_id, tipo_alarma, mensaje, timestamp, reconocida)
                        VALUES (?, ?, ?, ?, 0)
                    """, (sensor_id, tipo_alarma, mensaje, datetime.now().isoformat()))
                    
                    alarmas_creadas += 1
                    print(f"  🚨 Alarma creada para {nombre}: {mensaje}")
            
            conn.commit()
            print(f"  ✅ Total alarmas creadas: {alarmas_creadas}")
    
    def generar_reporte_alarmas():
        """Genera reporte completo de alarmas"""
        with get_db_connection() as conn:
            query_alarmas = """
            SELECT 
                s.nombre as sensor,
                s.ubicacion,
                a.tipo_alarma,
                a.mensaje,
                a.timestamp,
                a.reconocida,
                a.timestamp_reconocida,
                a.operador_id
            FROM alarmas a
            JOIN sensores s ON a.sensor_id = s.id
            WHERE a.timestamp >= datetime('now', '-24 hours')
            ORDER BY a.timestamp DESC
            """
            
            df_alarmas = pd.read_sql_query(query_alarmas, conn)
            
            print(f"  📋 Reporte de alarmas (últimas 24h):")
            print(f"    Total alarmas: {len(df_alarmas)}")
            print(f"    Alarmas pendientes: {len(df_alarmas[df_alarmas['reconocida'] == 0])}")
            print(f"    Alarmas reconocidas: {len(df_alarmas[df_alarmas['reconocida'] == 1])}")
            
            if len(df_alarmas) > 0:
                print("\n  🏆 Top 5 alarmas más recientes:")
                print(df_alarmas[['sensor', 'tipo_alarma', 'mensaje', 'reconocida']].head().to_string(index=False))
            
            return df_alarmas
    
    # Ejecutar detección y reporte
    detectar_alarmas_automaticas()
    df_alarmas = generar_reporte_alarmas()
    
    return df_alarmas

# Ejecutar ejercicios intermedios
df_analisis = ejercicio_3_analisis_pandas()
df_alarmas = ejercicio_4_gestion_alarmas()

print("\n✅ Ejercicios intermedios completados. ¡Avanza al nivel avanzado!")

### Nivel Avanzado 🔴

**Ejercicio 5: Optimización de Performance**
- Implementar índices en la base de datos
- Optimizar consultas complejas
- Medir y comparar tiempos de ejecución

**Ejercicio 6: Sistema de Respaldo Automático**
- Crear respaldo automático de la base de datos
- Implementar compresión de datos históricos
- Sistema de restauración de datos

### Proyecto Integrador 🏆

**Ejercicio 7: Dashboard Industrial Completo**
- Sistema completo de monitoreo industrial
- Integración de todos los componentes aprendidos
- Interfaz de usuario básica con reportes automáticos

In [ ]:
# 🔴 EJERCICIOS NIVEL AVANZADO Y PROYECTO INTEGRADOR

print("💪 EJERCICIOS PRÁCTICOS - NIVEL AVANZADO")
print("=" * 50)

# EJERCICIO 5: Optimización de Performance
def ejercicio_5_optimizacion():
    """
    EJERCICIO 5: Optimización de performance
    
    TAREAS:
    1. Crear índices para mejorar performance
    2. Medir tiempos de consultas
    3. Optimizar consultas complejas
    """
    print("\n🎯 EJERCICIO 5: Optimización de Performance")
    
    import time
    
    def medir_tiempo_consulta(consulta, parametros=None):
        """Mide el tiempo de ejecución de una consulta"""
        start_time = time.time()
        
        with get_db_connection() as conn:
            cursor = conn.cursor()
            if parametros:
                cursor.execute(consulta, parametros)
            else:
                cursor.execute(consulta)
            resultados = cursor.fetchall()
        
        end_time = time.time()
        return end_time - start_time, len(resultados)
    
    # 1. Crear índices para optimización
    def crear_indices_optimizacion():
        with get_db_connection() as conn:
            cursor = conn.cursor()
            
            # Índices para mejorar performance
            indices = [
                "CREATE INDEX IF NOT EXISTS idx_lecturas_timestamp ON lecturas(timestamp)",
                "CREATE INDEX IF NOT EXISTS idx_lecturas_sensor_timestamp ON lecturas(sensor_id, timestamp)",
                "CREATE INDEX IF NOT EXISTS idx_alarmas_timestamp ON alarmas(timestamp)",
                "CREATE INDEX IF NOT EXISTS idx_sensores_activo ON sensores(activo)",
                "CREATE INDEX IF NOT EXISTS idx_lecturas_calidad ON lecturas(calidad)"
            ]
            
            for indice in indices:
                cursor.execute(indice)
                print(f"    ✅ Índice creado: {indice.split(' ')[-1]}")
            
            conn.commit()
    
    # 2. Comparar performance antes y después
    consulta_compleja = """
    SELECT 
        s.nombre,
        s.tipo,
        COUNT(l.id) as total_lecturas,
        AVG(l.valor) as promedio,
        COUNT(CASE WHEN l.calidad = 'MALA' THEN 1 END) as lecturas_malas,
        MAX(l.timestamp) as ultima_lectura
    FROM sensores s
    LEFT JOIN lecturas l ON s.id = l.sensor_id
    WHERE s.activo = 1
    GROUP BY s.id, s.nombre, s.tipo
    ORDER BY total_lecturas DESC
    """
    
    print("  ⏱️ Midiendo performance de consulta compleja...")
    
    # Antes de índices (simulado - los índices ya están creados)
    tiempo_antes, registros = medir_tiempo_consulta(consulta_compleja)
    print(f"    Tiempo de consulta: {tiempo_antes:.4f} segundos")
    print(f"    Registros procesados: {registros}")
    
    crear_indices_optimizacion()
    
    # Después de índices
    tiempo_despues, _ = medir_tiempo_consulta(consulta_compleja)
    print(f"    Tiempo optimizado: {tiempo_despues:.4f} segundos")
    
    mejora = ((tiempo_antes - tiempo_despues) / tiempo_antes) * 100 if tiempo_antes > 0 else 0
    print(f"    🚀 Mejora de performance: {mejora:.1f}%")

# EJERCICIO 6: Sistema de Respaldo
def ejercicio_6_respaldo_automatico():
    """
    EJERCICIO 6: Sistema de respaldo automático
    
    TAREAS:
    1. Crear respaldo completo de la base de datos
    2. Comprimir datos históricos
    3. Implementar sistema de restauración
    """
    print("\n🎯 EJERCICIO 6: Sistema de Respaldo Automático")
    
    import shutil
    import zipfile
    import os
    
    def crear_respaldo_completo():
        """Crea un respaldo completo de la base de datos"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nombre_respaldo = f"respaldo_industrial_{timestamp}.db"
        
        try:
            # Copiar base de datos
            shutil.copy2('sistema_industrial.db', nombre_respaldo)
            
            # Comprimir respaldo
            nombre_zip = f"respaldo_industrial_{timestamp}.zip"
            with zipfile.ZipFile(nombre_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
                zipf.write(nombre_respaldo)
                
                # Agregar metadatos del respaldo
                metadatos = f"""
RESPALDO SISTEMA INDUSTRIAL
==========================
Fecha: {datetime.now().isoformat()}
Archivo: {nombre_respaldo}
Versión: 3.2.0

Este respaldo contiene:
- Tabla sensores
- Tabla lecturas
- Tabla alarmas
- Tabla operadores
- Todos los índices y triggers
                """
                
                zipf.writestr("metadatos_respaldo.txt", metadatos)
            
            # Limpiar archivo temporal
            os.remove(nombre_respaldo)
            
            print(f"    💾 Respaldo creado: {nombre_zip}")
            print(f"    📦 Tamaño: {os.path.getsize(nombre_zip) / 1024:.1f} KB")
            
            return nombre_zip
            
        except Exception as e:
            print(f"    ❌ Error en respaldo: {e}")
            return None
    
    def comprimir_datos_historicos():
        """Comprime datos históricos (simulación)"""
        with get_db_connection() as conn:
            cursor = conn.cursor()
            
            # Contar datos antiguos (más de 30 días)
            cursor.execute("""
                SELECT COUNT(*) FROM lecturas 
                WHERE timestamp < datetime('now', '-30 days')
            """)
            datos_antiguos = cursor.fetchone()[0]
            
            print(f"    📊 Datos antiguos identificados: {datos_antiguos} registros")
            
            if datos_antiguos > 0:
                # En un sistema real, aquí moveríamos los datos a una tabla comprimida
                print(f"    🗜️ Datos históricos candidatos para compresión")
            else:
                print(f"    ✅ No hay datos antiguos para comprimir")
    
    # Ejecutar respaldo
    archivo_respaldo = crear_respaldo_completo()
    comprimir_datos_historicos()
    
    return archivo_respaldo

# EJERCICIO 7: PROYECTO INTEGRADOR - Dashboard Industrial
def ejercicio_7_dashboard_completo():
    """
    EJERCICIO 7: PROYECTO INTEGRADOR - Dashboard Industrial Completo
    
    TAREAS:
    1. Sistema completo de monitoreo
    2. Reportes automáticos
    3. Gestión integral de alarmas
    4. Análisis predictivo básico
    """
    print("\n🎯 EJERCICIO 7: PROYECTO INTEGRADOR - Dashboard Industrial")
    print("🏆 SISTEMA COMPLETO DE MONITOREO INDUSTRIAL")
    print("=" * 60)
    
    class DashboardIndustrial:
        """Sistema completo de dashboard industrial"""
        
        def __init__(self):
            self.nombre_sistema = "SCADA Industrial v3.2"
            self.fecha_inicio = datetime.now()
        
        def estado_general(self):
            """Muestra el estado general del sistema"""
            print(f"\n🏭 {self.nombre_sistema}")
            print(f"📅 Sesión iniciada: {self.fecha_inicio.strftime('%Y-%m-%d %H:%M:%S')}")
            
            with get_db_connection() as conn:
                cursor = conn.cursor()
                
                # Estadísticas generales
                stats = {}
                
                cursor.execute("SELECT COUNT(*) FROM sensores WHERE activo = 1")
                stats['sensores_activos'] = cursor.fetchone()[0]
                
                cursor.execute("SELECT COUNT(*) FROM lecturas WHERE timestamp >= datetime('now', '-1 hour')")
                stats['lecturas_ultima_hora'] = cursor.fetchone()[0]
                
                cursor.execute("SELECT COUNT(*) FROM alarmas WHERE reconocida = 0")
                stats['alarmas_pendientes'] = cursor.fetchone()[0]
                
                cursor.execute("SELECT COUNT(DISTINCT operador_id) FROM alarmas WHERE timestamp >= datetime('now', '-24 hours')")
                stats['operadores_activos'] = cursor.fetchone()[0]
                
                # Mostrar dashboard
                print("\n📊 ESTADO DEL SISTEMA:")
                print(f"  🟢 Sensores activos: {stats['sensores_activos']}")
                print(f"  📈 Lecturas (1h): {stats['lecturas_ultima_hora']}")
                print(f"  🚨 Alarmas pendientes: {stats['alarmas_pendientes']}")
                print(f"  👥 Operadores activos: {stats['operadores_activos']}")
                
                return stats
        
        def analisis_predictivo(self):
            """Análisis predictivo básico"""
            print("\n🔮 ANÁLISIS PREDICTIVO:")
            
            with get_db_connection() as conn:
                # Análisis de tendencias por sensor
                query_tendencias = """
                SELECT 
                    s.nombre,
                    s.tipo,
                    AVG(l.valor) as promedio_24h,
                    COUNT(l.id) as frecuencia_lecturas
                FROM sensores s
                JOIN lecturas l ON s.id = l.sensor_id
                WHERE l.timestamp >= datetime('now', '-24 hours')
                GROUP BY s.id, s.nombre, s.tipo
                """
                
                df_tendencias = pd.read_sql_query(query_tendencias, conn)
                
                print("  📈 Sensores con mayor actividad:")
                top_sensores = df_tendencias.nlargest(3, 'frecuencia_lecturas')
                for _, sensor in top_sensores.iterrows():
                    print(f"    - {sensor['nombre']}: {sensor['frecuencia_lecturas']} lecturas (promedio: {sensor['promedio_24h']:.2f})")
                
                # Predicción simple de alarmas
                print("\n  🚨 Predicción de riesgos:")
                for _, sensor in df_tendencias.iterrows():
                    if sensor['frecuencia_lecturas'] < 5:
                        print(f"    ⚠️ {sensor['nombre']}: Baja frecuencia de datos - posible fallo")
        
        def generar_reporte_ejecutivo(self):
            """Genera reporte ejecutivo completo"""
            print("\n📋 REPORTE EJECUTIVO:")
            
            reporte = {
                'timestamp': datetime.now().isoformat(),
                'sistema': self.nombre_sistema,
                'estado': 'OPERACIONAL'
            }
            
            # Usar funciones previamente creadas
            datos_reporte = generar_reporte_operacional()
            
            print(f"  📊 Sistema: {reporte['sistema']}")
            print(f"  ✅ Estado: {reporte['estado']}")
            print(f"  🕐 Generado: {reporte['timestamp']}")
            
            return reporte
    
    # Ejecutar dashboard completo
    dashboard = DashboardIndustrial()
    stats = dashboard.estado_general()
    dashboard.analisis_predictivo()
    reporte = dashboard.generar_reporte_ejecutivo()
    
    print("\n🎉 ¡PROYECTO INTEGRADOR COMPLETADO!")
    print("Has dominado la integración de Python con SQLite para sistemas industriales.")
    
    return dashboard, stats, reporte

# Ejecutar ejercicios avanzados y proyecto
print("🚀 Ejecutando ejercicios avanzados...")

ejercicio_5_optimizacion()
archivo_respaldo = ejercicio_6_respaldo_automatico()
dashboard, stats, reporte = ejercicio_7_dashboard_completo()

print("\n" + "="*60)
print("🏆 TODOS LOS EJERCICIOS COMPLETADOS EXITOSAMENTE")
print("="*60)
print("✅ Nivel Básico: Conexiones y consultas")
print("✅ Nivel Intermedio: Análisis con Pandas y alarmas")  
print("✅ Nivel Avanzado: Optimización y respaldos")
print("✅ Proyecto Integrador: Dashboard industrial completo")
print("\n🎓 ¡Felicidades! Estás listo para el Módulo 3.3: ORM con SQLAlchemy")

## 8. 📝 Evaluación y Consolidación

### 8.1 Cuestionario Rápido de Evaluación

**¿Has completado todos los ejercicios?**
- [ ] Ejercicios básicos (conexión, consultas, inserción)
- [ ] Ejercicios intermedios (Pandas, alarmas)  
- [ ] Ejercicios avanzados (optimización, respaldos)
- [ ] Proyecto integrador (dashboard completo)

**Preguntas de comprensión:**

1. **¿Cuál es la ventaja de usar context managers (`with`) al trabajar con SQLite?**
   - a) Mayor velocidad de consultas
   - b) Gestión automática de conexiones y transacciones
   - c) Mejor compatibilidad con Pandas
   - d) Reduce el tamaño de la base de datos

2. **¿Qué patrón de diseño implementamos para la gestión de sensores?**
   - a) Singleton
   - b) Factory
   - c) DAO (Data Access Object)
   - d) Observer

3. **¿Cuál es la principal ventaja de usar `pd.read_sql_query()`?**
   - a) Es más rápido que SQLite puro
   - b) Convierte automáticamente los datos a DataFrame para análisis
   - c) Usa menos memoria
   - d) Es más seguro contra inyección SQL

4. **¿Para qué sirven los índices en SQLite?**
   - a) Reducir el tamaño de la base de datos
   - b) Mejorar la seguridad de los datos
   - c) Acelerar las consultas de búsqueda y filtrado
   - d) Hacer respaldos automáticos

5. **En un sistema industrial, ¿cuándo es crítico hacer un soft delete?**
   - a) Cuando el disco está lleno
   - b) Para mantener trazabilidad y auditoría
   - c) Para mejorar la velocidad
   - d) Solo en sistemas de producción

In [18]:
# 📝 AUTOEVALUACIÓN Y RESPUESTAS

def verificar_respuestas():
    """Verificación automática del cuestionario"""
    print("📊 VERIFICACIÓN DE RESPUESTAS")
    print("=" * 40)
    
    respuestas_correctas = {
        1: 'b',  # Context managers: gestión automática
        2: 'c',  # Patrón DAO  
        3: 'b',  # pd.read_sql_query: conversión a DataFrame
        4: 'c',  # Índices: acelerar consultas
        5: 'b'   # Soft delete: trazabilidad y auditoría
    }
    
    explicaciones = {
        1: "Los context managers garantizan que las conexiones se cierren automáticamente y las transacciones se manejen correctamente.",
        2: "El patrón DAO (Data Access Object) encapsula la lógica de acceso a datos, separando la lógica de negocio de la persistencia.",
        3: "pd.read_sql_query() convierte automáticamente los resultados SQL en DataFrames de Pandas, facilitando el análisis posterior.",
        4: "Los índices crean estructuras optimizadas para acelerar las operaciones de búsqueda, filtrado y ordenamiento.",
        5: "En sistemas industriales, el soft delete mantiene la trazabilidad histórica y cumple requisitos de auditoría y regulación."
    }
    
    print("✅ RESPUESTAS CORRECTAS:")
    for pregunta, respuesta in respuestas_correctas.items():
        print(f"\nPregunta {pregunta}: Opción {respuesta.upper()}")
        print(f"💡 Explicación: {explicaciones[pregunta]}")

verificar_respuestas()

# CHECKLIST DE CONSOLIDACIÓN DEL MÓDULO 3.2
def checklist_consolidacion():
    """Checklist completo de consolidación del módulo"""
    print("\n🎯 CHECKLIST DE CONSOLIDACIÓN - MÓDULO 3.2")
    print("=" * 60)
    
    checklist_items = [
        "✅ Configuración correcta del entorno (Python + SQLite + Pandas)",
        "✅ Comprensión de conexiones y context managers",
        "✅ Dominio de operaciones CRUD (Create, Read, Update, Delete)",
        "✅ Implementación de esquemas industriales realistas",
        "✅ Integración efectiva de SQLite con Pandas",
        "✅ Análisis estadístico de datos industriales",
        "✅ Sistema completo de gestión de alarmas",
        "✅ Implementación del patrón DAO",
        "✅ Optimización de consultas con índices",
        "✅ Sistema de respaldos automáticos",
        "✅ Generación de reportes automáticos",
        "✅ Exportación de datos a Excel",
        "✅ Detección de valores atípicos y anomalías",
        "✅ Dashboard industrial integrado",
        "✅ Manejo profesional de errores y excepciones",
        "✅ Implementación de buenas prácticas de seguridad"
    ]
    
    print("📋 CONOCIMIENTOS CONSOLIDADOS:")
    for item in checklist_items:
        print(f"  {item}")
    
    print(f"\n🏆 TOTAL OBJETIVOS COMPLETADOS: {len(checklist_items)}/16")
    print("\n🎓 NIVEL DE COMPETENCIA: AVANZADO")
    print("📈 PREPARACIÓN PARA MÓDULO 3.3: 100% LISTO")

checklist_consolidacion()

print("\n" + "="*60)
print("🎉 ¡MÓDULO 3.2 COMPLETAMENTE CONSOLIDADO!")
print("="*60)
print("🚀 Siguiente etapa: Módulo 3.3 - ORM con SQLAlchemy")
print("📚 Ya tienes las bases sólidas para frameworks avanzados de ORM")
print("💪 ¡Excelente trabajo siguiendo la metodología de aprendizaje deliberado!")

📊 VERIFICACIÓN DE RESPUESTAS
✅ RESPUESTAS CORRECTAS:

Pregunta 1: Opción B
💡 Explicación: Los context managers garantizan que las conexiones se cierren automáticamente y las transacciones se manejen correctamente.

Pregunta 2: Opción C
💡 Explicación: El patrón DAO (Data Access Object) encapsula la lógica de acceso a datos, separando la lógica de negocio de la persistencia.

Pregunta 3: Opción B
💡 Explicación: pd.read_sql_query() convierte automáticamente los resultados SQL en DataFrames de Pandas, facilitando el análisis posterior.

Pregunta 4: Opción C
💡 Explicación: Los índices crean estructuras optimizadas para acelerar las operaciones de búsqueda, filtrado y ordenamiento.

Pregunta 5: Opción B
💡 Explicación: En sistemas industriales, el soft delete mantiene la trazabilidad histórica y cumple requisitos de auditoría y regulación.

🎯 CHECKLIST DE CONSOLIDACIÓN - MÓDULO 3.2
📋 CONOCIMIENTOS CONSOLIDADOS:
  ✅ Configuración correcta del entorno (Python + SQLite + Pandas)
  ✅ Comprensión 

In [12]:
# 🚀 DEMO RÁPIDA - CREAR DATOS DE PRUEBA
def demo_rapida_sistema_industrial():
    """Crea un sistema industrial simplificado para demostración"""
    
    # Crear nueva base de datos limpia
    conn = sqlite3.connect('sistema_industrial.db')
    cursor = conn.cursor()
    
    print("🏭 CREANDO SISTEMA INDUSTRIAL DE DEMOSTRACIÓN")
    print("=" * 50)
    
    # Crear tabla de sensores simplificada
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS sensores (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT UNIQUE NOT NULL,
        tipo TEXT NOT NULL,
        ubicacion TEXT NOT NULL,
        rango_min REAL DEFAULT 0,
        rango_max REAL DEFAULT 100,
        activo INTEGER DEFAULT 1
    )
    ''')
    
    # Crear tabla de lecturas
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS lecturas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        sensor_id INTEGER,
        valor REAL NOT NULL,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
        calidad TEXT DEFAULT 'BUENA',
        FOREIGN KEY (sensor_id) REFERENCES sensores (id)
    )
    ''')
    
    # Crear tabla de alarmas
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS alarmas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        sensor_id INTEGER,
        tipo_alarma TEXT NOT NULL,
        mensaje TEXT NOT NULL,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
        reconocida INTEGER DEFAULT 0,
        FOREIGN KEY (sensor_id) REFERENCES sensores (id)
    )
    ''')
    
    # Insertar sensores de ejemplo
    sensores_demo = [
        ('TEMP_REACTOR_01', 'temperatura', 'Reactor Principal', 50, 300),
        ('PRES_BOMBA_A1', 'presion', 'Bomba A1', 0, 50),
        ('FLUJO_LINEA_3', 'flujo', 'Línea Producción 3', 10, 200),
        ('NIVEL_TANQUE_B', 'nivel', 'Tanque B', 0, 100),
        ('TEMP_MOTOR_M2', 'temperatura', 'Motor M2', 20, 80)
    ]
    
    cursor.executemany('''
        INSERT OR IGNORE INTO sensores (nombre, tipo, ubicacion, rango_min, rango_max)
        VALUES (?, ?, ?, ?, ?)
    ''', sensores_demo)
    
    print(f"✅ {len(sensores_demo)} sensores creados")
    
    # Generar lecturas realistas
    import random
    from datetime import datetime, timedelta
    
    lecturas_demo = []
    for sensor_id in range(1, 6):  # 5 sensores
        for i in range(20):  # 20 lecturas por sensor
            timestamp = datetime.now() - timedelta(hours=random.randint(0, 72))
            
            # Generar valores realistas según el tipo
            if sensor_id == 1:  # Temperatura reactor
                valor = random.uniform(150, 280)
            elif sensor_id == 2:  # Presión bomba
                valor = random.uniform(15, 45)
            elif sensor_id == 3:  # Flujo línea
                valor = random.uniform(50, 180)
            elif sensor_id == 4:  # Nivel tanque
                valor = random.uniform(20, 95)
            else:  # Temperatura motor
                valor = random.uniform(35, 70)
            
            calidad = random.choice(['BUENA', 'BUENA', 'BUENA', 'REGULAR', 'MALA'])
            
            lecturas_demo.append((sensor_id, round(valor, 2), timestamp.isoformat(), calidad))
    
    cursor.executemany('''
        INSERT INTO lecturas (sensor_id, valor, timestamp, calidad)
        VALUES (?, ?, ?, ?)
    ''', lecturas_demo)
    
    print(f"✅ {len(lecturas_demo)} lecturas generadas")
    
    # Generar algunas alarmas de ejemplo
    alarmas_demo = [
        (1, 'TEMPERATURA_ALTA', 'Temperatura reactor por encima de 280°C'),
        (2, 'PRESION_BAJA', 'Presión bomba por debajo de 15 bar'),
        (3, 'FLUJO_IRREGULAR', 'Fluctuaciones anormales en flujo'),
    ]
    
    cursor.executemany('''
        INSERT INTO alarmas (sensor_id, tipo_alarma, mensaje)
        VALUES (?, ?, ?)
    ''', alarmas_demo)
    
    print(f"✅ {len(alarmas_demo)} alarmas creadas")
    
    conn.commit()
    conn.close()
    
    print("🎉 Sistema industrial de demostración listo!")
    return 'sistema_industrial.db'

# Crear sistema de demostración
db_demo = demo_rapida_sistema_industrial()

# Actualizar función de conexión para usar la nueva base
def get_db_connection():
    """Context manager para el sistema industrial"""
    import contextlib
    
    @contextlib.contextmanager
    def db_connection():
        conn = None
        try:
            conn = sqlite3.connect('sistema_industrial.db')
            conn.row_factory = sqlite3.Row
            yield conn
            conn.commit()
        except Exception as e:
            if conn:
                conn.rollback()
            raise e
        finally:
            if conn:
                conn.close()
    
    return db_connection()

print("\n🔧 Función de conexión actualizada para sistema industrial")

🏭 CREANDO SISTEMA INDUSTRIAL DE DEMOSTRACIÓN
✅ 5 sensores creados
✅ 100 lecturas generadas
✅ 3 alarmas creadas
🎉 Sistema industrial de demostración listo!

🔧 Función de conexión actualizada para sistema industrial
